In [ ]:
!pip install -q docling docling-core

In [1]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
import time

## 1. Khởi tạo Docling converter

In [15]:
# Define pipeline options for PDF processing
pipeline_options = PdfPipelineOptions(
    do_table_structure=True,  # Enable table structure detection
    do_ocr=False,  # Enable OCR
    # full page ocr and language selection
    #ocr_options=EasyOcrOptions(force_full_page_ocr=True, lang=["en"]),  # Use EasyOCR for OCR
    # ocr_options=TesseractOcrOptions(force_full_page_ocr=True, lang=["eng"]),  # Uncomment to use Tesseract for OCR
    #ocr_options = OcrMacOptions(force_full_page_ocr=True, lang=['en-US']),
    table_structure_options=dict(
        do_cell_matching=False,  # Use text cells predicted from table structure model
        mode=TableFormerMode.ACCURATE # Use more accurate Tableformer model
    ),
    generate_page_images=False,  # Enable page image generation
    generate_picture_images=False,  # Enable picture image generation
    # images_scale=IMAGE_RESOLUTION_SCALE,  # Set image resolution scale (scale=1 corresponds to a standard 72 DPI image)
)


format_options = {
    InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
}
converter = DocumentConverter(format_options=format_options)

def convert_pdf_to_docling(sources: list[str]):
    start_time = time.time()
    conversions = {
        source: converter.convert(source).document for source in sources
    }
    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.4f}")
    return conversions

In [17]:
sources = [
    "CA Warn Report.pdf",
    # "/content/drive/MyDrive/Docling/national-capitals.pdf",
    # "/content/drive/MyDrive/Docling/c935e2902adf7040a6ffe0db0f7c11e6.pdf",
]

conversions = convert_pdf_to_docling(sources)

Time taken: 316.2556


## 2. Các hàm hữu ích

In [21]:
import csv
import pandas as pd
from io import StringIO
import numpy as np


def preprocess_df(df):
    # Check if columns are numeric sequence
    if all(str(col).isdigit() for col in df.columns):
        # Get first row as new columns
        new_cols = df.iloc[0].values
        # Drop first row and reset columns
        df = df.iloc[1:].reset_index(drop=True)
        df.columns = new_cols
        
    # Check if first row is numeric sequence
    elif df.shape[0] > 0:
        first_row = df.iloc[0]
        try:
            # Convert to numeric and check if it's a sequence
            nums = pd.to_numeric(first_row)
            if all(nums[i] == i for i in range(len(nums))):
                # Drop first row if it's a numeric sequence
                df = df.iloc[1:].reset_index(drop=True)
        except:
            pass
            
    return df



def _perform_csv_sniffing(csv_sample_string: str) -> bool:
    """Kiểm tra sự tồn tại của header trong chuỗi CSV mẫu bằng csv.Sniffer."""
    if not csv_sample_string.strip():
        return False
    sniffer = csv.Sniffer()
    try:
        return sniffer.has_header(csv_sample_string)
    except csv.Error: # Lỗi định dạng CSV không xác định được
        return False

def has_header(data_source: any) -> bool:
    """
    Kiểm tra DataFrame (hoặc file CSV) có header không.
    Trả về: True nếu có header, False nếu không.
    """
    MAX_SAMPLE_ROWS_FOR_DF = 15

    if isinstance(data_source, pd.DataFrame):
        df = data_source
        if df.empty:
            return False

        columns = df.columns.tolist()
        if not columns: # Không có cột, không có header
            return False

        # Loại trừ cột là số nguyên tuần tự (0,1,2...) làm header
        is_sequential_integers = True
        for i, col_name_obj in enumerate(columns):
            try:
                if int(str(col_name_obj)) != i:
                    is_sequential_integers = False
                    break
            except ValueError:
                is_sequential_integers = False
                break
        if is_sequential_integers:
            return False

        # Loại trừ cột 'Unnamed: X' làm header
        for col in columns:
            if str(col).startswith('Unnamed'):
                return False
        
        # Dùng CSV sniffing trên mẫu DataFrame
        sample_df = df.head(MAX_SAMPLE_ROWS_FOR_DF)
        
        # (Nếu DataFrame chỉ có tên cột đã qua kiểm tra, vẫn coi là có header; logic tiếp tục)

        csv_string_buffer = StringIO()
        # Chuyển mẫu DataFrame (kèm header) sang CSV để sniff
        sample_df.to_csv(csv_string_buffer, header=True, index=False)
        csv_sample_from_df = csv_string_buffer.getvalue()
        
        return _perform_csv_sniffing(csv_sample_from_df)

    elif isinstance(data_source, str): # Xử lý trường hợp data_source là đường dẫn file CSV
        MAX_FILE_LINES_FOR_SAMPLE = 15 # Giới hạn số dòng đọc cho mẫu file
        try:
            with open(data_source, 'r', newline='', encoding='utf-8-sig') as csv_file:
                sample_lines_list = [line for i, line in enumerate(csv_file) if i < MAX_FILE_LINES_FOR_SAMPLE]
                if not sample_lines_list:
                    return False
                return _perform_csv_sniffing("".join(sample_lines_list))
        except FileNotFoundError:
            return False 
        except Exception: # Lỗi đọc file khác
            return False
    else: # Kiểu dữ liệu không được hỗ trợ
        return False

def solve_non_header_table(df: pd.DataFrame, headers: list) -> pd.DataFrame:
    """
    Chuyển đổi DataFrame không có header: tên cột cũ thành dòng dữ liệu đầu tiên, áp dụng `headers` mới.
    Cột 'Unnamed' cũ thành NaN trong dòng dữ liệu mới.

    Args:
        df (pd.DataFrame): DataFrame đầu vào (không có header).
        headers (list): Danh sách header mới.
    Returns:
        pd.DataFrame: DataFrame đã điều chỉnh.
    """
    df_copy = df.copy() # Làm việc trên bản sao

    first_row_values = []
    for col_original_name in df_copy.columns:
        if str(col_original_name).startswith('Unnamed'):
            first_row_values.append(np.nan)
        else:
            first_row_values.append(col_original_name) # Tên cột cũ thành dữ liệu
            
    # Đảm bảo first_row_values khớp độ dài headers.
    # Logic gọi hàm main_concatenation_logic thường đảm bảo len(df.columns) == len(headers).
    if len(first_row_values) != len(headers):
        if len(first_row_values) > len(headers): # Cắt bớt nếu thừa
            first_row_values = first_row_values[:len(headers)]
        else: # Thêm NaN nếu thiếu
            first_row_values.extend([np.nan] * (len(headers) - len(first_row_values)))

    temp_firstrow_df = pd.DataFrame([first_row_values], columns=headers) # Dòng dữ liệu mới từ tên cột cũ
    df_copy.columns = headers # Áp dụng header mới cho dữ liệu gốc
    
    result_df = pd.concat([temp_firstrow_df, df_copy], ignore_index=True) # Ghép lại
    return result_df.reset_index(drop=True)

def get_input_df(df, n_rows, sep="="):
        columns = df.columns.tolist()
        # Check if any columns start with "Unnamed" or are numeric indices
        unnamed_cols = [col for col in columns if str(col).startswith("Unnamed")]
        # check columns is all digits
        numeric_cols = all(
            isinstance(col, int) or (isinstance(col, str) and col.isdigit())
            for col in columns
        )
        # check if NAN exists in columns
        nan_cols = any(pd.isna(col) for col in df.columns)
        header = not (
            bool(unnamed_cols) or numeric_cols or nan_cols or not has_header(df)
        )  # Don't show header if unnamed or numeric columns or has NaN columns

        if len(df) < n_rows:
            if header:
                return f"Headers: {','.join(columns)}\n" + df.to_csv(
                    index=False, header=False, sep=sep
                )
            return df.to_csv(index=False, header=False, sep=sep)
        else:
            # Calculate number of rows for each section
            n_edge = n_rows // 3  # Number of rows to take from start and end
            n_random = n_rows - (2 * n_edge)  # Remaining rows will be random

            # Get first and last n_edge rows
            first_rows = df.head(n_edge)
            last_rows = df.tail(n_edge)
            # Get random rows from middle section
            middle_section = df.iloc[n_edge:-n_edge]
            random_rows = (
                middle_section.sample(n=n_random)
                if len(middle_section) >= n_random
                else middle_section
            )

            concated_df = pd.concat([first_rows, last_rows, random_rows])
            if header:
                return f"Headers: {','.join(columns)}\n" + concated_df.to_csv(
                    index=False, header=False, sep=sep
                )
            return concated_df.to_csv(index=False, header=False, sep=sep)

def get_column_types(df):
    types = []
    if df.columns.empty:
        return types

    for col_name in df.columns: 
        current_column_object = df[col_name]

        is_truly_all_na = False
        effective_dtype = None

        if isinstance(current_column_object, pd.DataFrame):
            is_truly_all_na = pd.isna(current_column_object).all().all()
            if not is_truly_all_na:
                effective_dtype = current_column_object.iloc[:, 0].dtype
        elif isinstance(current_column_object, pd.Series):
            is_truly_all_na = pd.isna(current_column_object).all()
            if not is_truly_all_na:
                effective_dtype = current_column_object.dtype
        else:
            types.append("error_unexpected_type")
            continue

        if is_truly_all_na:
            types.append("unknown")
        else:
            if effective_dtype is not None:
                types.append(
                    "numeric" if pd.api.types.is_numeric_dtype(effective_dtype) else str(effective_dtype)
                )
            else:
                types.append("unknown_error_in_logic")
    return types

## 3. Xử lý

Duyệt qua Docling và lấy các tables dưới dạng `pd.DataFrame` và tạo ra `input_prompt` cho LLM

In [29]:
from docling_core.types.doc.labels import DocItemLabel
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.document import TableItem
from langchain_core.documents import Document
import pandas as pd

def get_tables_from_docling(docling_document) -> tuple[list[pd.DataFrame], str]:
    doc_id = 0
    tables: list[pd.DataFrame] = []
    full_prompt = ""
    for table in docling_document.tables:
        if table.label in [DocItemLabel.TABLE]:
            # ref = table.get_ref().cref
            # text = table.export_to_markdown(docling_document)
            df = preprocess_df(table.export_to_dataframe())
            df_string = get_input_df(df, n_rows=10)
            column_types = get_column_types(df=df)
            n_columns = len(df.columns.to_list())
            full_prompt += f"Table {doc_id}\n{df_string}\nNumber of columns: {n_columns}\nColumn types: {column_types}\n\n"
            tables.append(df)
            doc_id += 1

    print(f"{len(tables)} tables extracted")
    return tables, full_prompt

In [32]:
tables, full_prompt = get_tables_from_docling(conversions[sources[0]])

17 tables extracted


### Hàm call LLM API

In [34]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
import json
from google import genai
from google.genai import types
# from google.colab import userdata
# GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
from dotenv import load_dotenv
load_dotenv()
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY')

def generate(prompt):
    model = "gemini-2.0-flash-lite"
    client = genai.Client(
        api_key=GEMINI_API_KEY,
    )
    system_prompt = """You are an AI assistant specialized in analyzing and grouping data tables for vertical concatenation. I will provide you with information for multiple tables. Each table will include:

Table index (e.g., \"Table 0\").
Explicit headers (e.g., \"Headers: ColA,ColB,ColC\") if this table introduces a new header structure. For subsequent tables with the same structure and headers, the \"Headers:\" line can be omitted.
A few lines of sample data. Each data line contains values separated by the \"=\" character.
The number of columns.
A list of data types for the columns.
The tables will be presented sequentially.

Your task is to identify groups of tables that can be vertically concatenated. Tables are considered concatenable if they share the following characteristics:

The same number of columns.
Compatible data types for corresponding columns.
Identical or very similar headers. If headers are not explicitly provided for a table, assume it shares the headers of the immediately preceding table (or the closest table with explicit headers) if their number of columns and data types match.
Below is the table data:"""
    content = system_prompt + prompt
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
            type = genai.types.Type.OBJECT,
            required = ["concatable_tables"],
            properties = {
                "concatable_tables": genai.types.Schema(
                    type = genai.types.Type.ARRAY,
                    items = genai.types.Schema(
                        type = genai.types.Type.OBJECT,
                        required = ["table_index"],
                        properties = {
                            "table_index": genai.types.Schema(
                                type = genai.types.Type.ARRAY,
                                items = genai.types.Schema(
                                    type = genai.types.Type.NUMBER,
                                ),
                            ),
                        },
                    ),
                ),
            },
        ),
        system_instruction=[
            types.Part.from_text(text=system_prompt),
        ],
    )
    input_tokens = client.models.count_tokens(model=model, contents=[content])
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )
    output_tokens = client.models.count_tokens(model=model, contents=response.text)
    print(f"Input tokens: {input_tokens}")
    print(f"Output tokens: {output_tokens}")

    json_response = json.loads(response.text)

    return json_response


In [35]:
json_instructions = generate(full_prompt)
print(json_instructions)

Input tokens: total_tokens=6494 cached_content_token_count=None
Output tokens: total_tokens=123 cached_content_token_count=None
{'concatable_tables': [{'table_index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]}, {'table_index': [15, 16]}]}


Ghép nối các nhóm tables lại với nhau

In [36]:
def main_concatenation_logic(tables: list[pd.DataFrame], concat_json: dict) -> list[pd.DataFrame]:
    """
    Ghép các DataFrame theo nhóm chỉ định trong `concat_json`.
    Sử dụng `has_header` và `solve_non_header_table` để chuẩn hóa header.

    Args:
        tables (list[pd.DataFrame]): Danh sách tất cả DataFrame.
        concat_json (dict): Hướng dẫn ghép bảng, ví dụ:
                            {'concatable_tables': [{'table_index': [0, 1]}, ...]}
    Returns:
        list[pd.DataFrame]: Danh sách các DataFrame đã ghép.
    """
    concatenated_results = [] # Lưu kết quả ghép
    
    # Kiểm tra đầu vào cơ bản
    if not isinstance(tables, list) or not all(isinstance(t, pd.DataFrame) for t in tables):
        print("Lỗi: 'tables' phải là list[pd.DataFrame].") # Cân nhắc dùng raise ValueError
        return [] 
    if not isinstance(concat_json, dict) or 'concatable_tables' not in concat_json:
        print("Lỗi: 'concat_json' không hợp lệ hoặc thiếu 'concatable_tables'.") # Cân nhắc dùng raise ValueError
        return []

    for group_instruction in concat_json.get('concatable_tables', []): # Xử lý từng nhóm ghép
        table_indices = group_instruction.get('table_index', [])
        if not table_indices: # Bỏ qua nhóm rỗng
            continue

        # Tập hợp các DataFrame trong nhóm hiện tại
        current_group_dfs_to_process = []
        for index in table_indices:
            if 0 <= index < len(tables):
                current_group_dfs_to_process.append(tables[index])
            # else: Bỏ qua chỉ mục ngoài giới hạn (không thông báo để code ngắn gọn)

        if not current_group_dfs_to_process:
            continue
            
        # Xác định header chuẩn cho nhóm (nếu có)
        authoritative_headers = None
        for df_check in current_group_dfs_to_process:
            if has_header(df_check): 
                authoritative_headers = df_check.columns.tolist()
                break
        
        processed_dfs_for_this_group = [] # Các DataFrame đã xử lý, sẵn sàng để ghép
        for original_df_in_group in current_group_dfs_to_process:
            df_to_process = original_df_in_group.copy() # Luôn xử lý bản sao

            if authoritative_headers is not None: # Trường hợp nhóm có header chuẩn
                if has_header(df_to_process): # Bảng hiện tại có header
                    # Nếu cùng số cột, đổi tên theo header chuẩn để đồng nhất
                    if len(df_to_process.columns) == len(authoritative_headers):
                        df_to_process.columns = authoritative_headers
                    # else: Khác số cột, giữ header gốc, pd.concat sẽ tự xử lý việc hợp nhất cột
                else: # Bảng hiện tại không có header
                    # Nếu cùng số cột, áp dụng logic "sửa" header
                    if len(df_to_process.columns) == len(authoritative_headers):
                        df_to_process = solve_non_header_table(df_to_process, authoritative_headers)
                    # else: Khác số cột, không xử lý, pd.concat sẽ tự xử lý
            # else: Trường hợp nhóm không có header chuẩn -> ghép các bảng nguyên trạng
            
            processed_dfs_for_this_group.append(df_to_process)

        if processed_dfs_for_this_group: # Thực hiện ghép nếu có bảng đã xử lý
            final_df = pd.concat(processed_dfs_for_this_group, ignore_index=True)
            concatenated_results.append(final_df)
            
    return concatenated_results

### Kết quả

In [37]:
concatenated_dataframes = main_concatenation_logic(tables, json_instructions)
print(f"Có tổng cộng {len(concatenated_dataframes)} bảng!")

Có tổng cộng 2 bảng!


In [38]:
from IPython.display import display, Markdown

processed_path = "processed_df"

for i, df_result in enumerate(concatenated_dataframes):
    print(f"\n--- Concatenated DataFrame Group {i+1} ---")
    display(Markdown(df_result.to_markdown()))
    print(f"Columns: {df_result.columns.tolist()}")
    print(f"Shape: {df_result.shape}")
    with open(f"{processed_path}/{i}.txt", "w") as f:
        f.write(df_result.to_string(index=False))


--- Concatenated DataFrame Group 1 ---


|     | Notice Date   | Effective    | Received    | Company                                               | City              |   No. Of  | Layoff/Closure               |
|----:|:--------------|:-------------|:------------|:------------------------------------------------------|:------------------|----------:|:-----------------------------|
|   0 |               | 03/25/20     | 07/01/20    | Maxim Integrated Product                              | San Jose          |       150 | Closure Permanent            |
|   1 |               | 08/29/20     | 07/01/20    | McGraw-Hill Education                                 | Monterey          |       137 | Layoff Unknown at this time  |
|   2 |               | 08/30/20     | 07/01/20    | Long Beach Memorial Medical Center                    | Long Beach        |        90 | Layoff Permanent             |
|   3 |               | 09/02/20     | 07/01/20    | Leidos                                                | El Segundo        |        72 | Layoff Permanent             |
|   4 |               | 09/30/20     | 07/01/20    | Bosch Healthcare Systems, Inc.                        | Palo Alto         |        55 | Closure Permanent            |
|   5 |               | 09/01/20     | 07/02/20    | Encompass Digital Media, Inc.                         | Los Angeles       |        41 | Closure Permanent            |
|   6 |               | 07/06/20     | 07/02/20    | Alphatec Spine                                        | Carlsbad          |        99 | Layoff Permanent             |
|   7 |               | 08/07/20     | 07/06/20    | Symantec Corporation                                  | Mountain View     |        60 | Layoff Permanent             |
|   8 |               | 08/31/20     | 07/06/20    | Fusion Contacts Centers, LLC                          | Santa Maria       |        50 | Closure Permanent            |
|   9 |               | 09/15/20     | 07/06/20    | KLA-Tencor Corporation                                | Milpitas          |       213 | Layoff Permanent             |
|  10 |               | 09/04/20     | 07/06/20    | Southern California Edison Company                    | San Clemente      |       100 | Closure Permanent            |
|  11 |               | 09/01/20     | 07/06/20    | State Fish Company, Inc.                              | Wilmington        |        76 | Closure Permanent            |
|  12 |               | 09/04/20     | 07/06/20    | Boeing Company                                        | Long Beach        |        56 | Layoff Unknown at this time  |
|  13 |               | 09/04/20     | 07/06/20    | Bridgepoint Education, Inc.                           | San Diego         |         7 | Layoff Permanent             |
|  14 |               | 09/04/20     | 07/06/20    | Bridgepoint Education, Inc.                           | San Diego         |        15 | Layoff Permanent             |
|  15 |               | 06/29/20     | 07/07/20    | BAE Systems                                           | San Francisco     |         4 | Layoff Temporary             |
|  16 |               | 06/29/20     | 07/07/20    | BAE Systems                                           | San Francisco     |        78 | Layoff Temporary             |
|  17 |               | 09/07/20     | 07/07/20    | Bay Bread LLC dba Bakery Los Angeles                  | San Fernando      |        50 | Closure Permanent            |
|  18 |               | 09/25/20     | 07/07/20    | Bay Bread LLC dba New French Bakery                   | South San         |       121 | Closure Permanent            |
|  19 |               | 06/12/20     | 07/07/20    | Hewlett-Packard Company                               | Palo Alto         |        65 | Layoff Permanent             |
|  20 |               | 09/06/20     | 07/08/20    | Microsoft Corporation                                 | San Diego         |       129 | Layoff Permanent             |
|  21 |               | 10/09/20     | 07/10/20    | Aramark Healthcare Support Services,                  | Culver City       |        53 | Closure Permanent            |
|  22 |               | 09/10/20     | 07/10/20    | Maxim Integrated Product                              | San Jose          |        20 | Layoff Permanent             |
|  23 |               | 09/04/20     | 07/10/20    | ProCourier, Inc.                                      | San Diego         |        22 | Layoff Unknown at this time  |
|  24 |               | 09/04/20     | 07/10/20    | ProCourier, Inc.                                      | Los Angeles       |        71 | Layoff Unknown at this time  |
|  25 |               | 09/04/20     | 07/10/20    | ProCourier, Inc.                                      | Irvine            |        22 | Layoff Unknown at this time  |
|  26 |               | 07/22/20     | 07/10/20    | Berkeley Pyramid Alehouse                             | Berkeley          |        63 | Closure Permanent            |
|  27 |               | 09/14/20     | 07/10/20    | Fireman's Fund Insurance Company                      | Novato            |        35 | Layoff Permanent             |
|  28 |               | 08/31/20     | 07/13/20    | First Transit                                         | San Bernardino    |       127 | Layoff Permanent             |
|  29 |               | 08/31/20     | 07/13/20    | First Transit                                         | Rancho            |        71 | Layoff Permanent             |
|  30 |               | 07/14/20     | 07/13/20    | 11 Main LLC                                           | San Mateo         |        35 | Closure Permanent            |
|  31 |               | 07/14/20     | 07/13/20    | 11 Main LLC                                           | Chico             |        44 | Layoff Permanent             |
|  32 |               | 07/15/20     | 07/15/20    | TaylorMade Golf Company                               | Carlsbad          |        64 | Layoff Permanent             |
|  33 |               | 09/06/20     | 07/16/20    | Southern California Edison Company                    | Rosemead          |        38 | Layoff Permanent             |
|  34 |               | 09/18/20     | 07/20/20    | Actavis, Inc.                                         | Corona            |        45 | Layoff Permanent             |
|  35 |               | 07/13/20     | 07/21/20    | American Management Services LLC                      | Monterey          |        56 | Closure Permanent            |
|  36 |               | 09/18/20     | 07/21/20    | Boeing Company                                        | Huntington Beach  |        65 | Layoff Unknown at this time  |
|  37 |               | 09/18/20     | 07/21/20    | Boeing Company                                        | Long Beach        |       113 | Layoff Unknown at this time  |
|  38 |               | 09/18/20     | 07/21/20    | Boeing Company                                        | El Segundo        |        66 | Layoff Unknown at this time  |
|  39 |               | 09/25/20     | 07/21/20    | Maxim Integrated Product                              | San Jose          |        11 | Layoff Permanent             |
|  40 |               | 09/26/20     | 07/21/20    | DS Services of America, Inc.                          | Los Angeles       |       120 | Closure Permanent            |
|  41 |               | 09/26/20     | 07/21/20    | DS Services of America, Inc.                          | Pasadena          |       200 | Closure Permanent            |
|  42 |               | 09/18/20     | 07/21/20    | Minno, Inc.                                           | San Francisco     |       255 | Closure Permanent            |
|  43 |               | 09/18/20     | 07/21/20    | Safeway, Inc.                                         | Pleasanton        |        14 | Layoff Unknown at this time  |
|  44 |               | 09/15/20     | 07/22/20    | Oakley, Inc.                                          | Foothill Ranch    |       137 | Layoff Unknown at this time  |
|  45 |               | 09/15/20     | 07/22/20    | Oakley, Inc.                                          | Lake Forest       |        22 | Layoff Unknown at this time  |
|  46 |               | 09/15/20     | 07/22/20    | Oakley, Inc.                                          | Encinitas         |         8 | Layoff Unknown at this time  |
|  47 |               | 12/31/20     | 07/22/20    | Toyota Motor North America                            | Gardena           |       100 | Closure Permanent            |
|  48 |               | 12/31/20     | 07/22/20    | Toyota Motor North America                            | Torrance          |      3000 | Closure Permanent            |
|  49 |               | 07/22/20     | 07/22/20    | Cisco Systems, Inc.                                   | San Jose          |       192 | Layoff Permanent             |
|  50 |               | 09/30/20     | 07/23/20    | IAP Worldwide Services, Inc.                          | Mountain View     |        67 | Layoff Permanent             |
|  51 |               | 09/15/20     | 07/23/20    | Presse LLC dba Cafe de la Presse and                  | San Francisco     |        85 | Closure Temporary            |
|  52 |               | 10/04/20     | 07/24/20    | Sears Holdings Corporation                            | Cupertino         |        81 | Closure Permanent            |
|  53 |               | 10/04/20     | 07/24/20    | Sears Holdings Corporation                            | Cupertino         |        17 | Closure Permanent            |
|  54 |               | 09/14/20     | 07/24/20    | Associate Management Resources, Inc.                  | San Diego         |       533 | Closure Permanent            |
|  55 |               | 10/11/20     | 07/27/20    | Kmart                                                 | Atwater           |        74 | Closure Permanent            |
|  56 |               | 09/25/20     | 07/27/20    | Safeway, Inc.                                         | Pleasanton        |        37 | Layoff Unknown at this time  |
|  57 |               | 09/30/20     | 07/29/20    | Hank Fisher Properties, Inc. - Chateau at  Carmichael |                   |        57 | Layoff Permanent             |
|  58 |               | 09/30/20     | 07/29/20    | Hank Fisher Properties, Inc. - Chateau                | Sacramento        |        42 | Layoff Permanent             |
|  59 |               | 09/30/20     | 07/29/20    | Hank Fisher Properties, Inc. - River's                | Sacramento        |        34 | Layoff Permanent             |
|  60 |               | 09/30/20     | 07/29/20    | Hank Fisher Properties, Inc. - The                    | Sacramento        |        89 | Layoff Permanent             |
|  61 |               | 10/15/20     | 07/29/20    | Hank Fisher Properties, Inc.                          | Sacramento        |         7 | Layoff Permanent             |
|  62 |               | 12/31/20     | 07/29/20    | Hank Fisher Properties, Inc. - Leisure                | Sacramento        |        13 | Layoff Permanent             |
|  63 |               | 09/30/20     | 07/29/20    | SPS Distribution Center                               | Paso Robles       |         8 | Closure Permanent            |
|  64 |               | 07/31/20     | 07/30/20    | Turf Terminators, LLC                                 | Los Angeles       |        94 | Layoff Permanent             |
|  65 |               | 07/28/20     | 07/30/20    | Relativity Media                                      | Beverly Hills     |        33 | Layoff Permanent             |
|  66 |               | 07/28/20     | 07/30/20    | Relativity Media                                      | Beverly Hills     |        28 | Layoff Permanent             |
|  67 |               | 09/28/20     | 07/30/20    | Buca Restaurants 2, Inc.(CANCELLED)** Santa Monica    |                   |        61 | Closure Permanent            |
|  68 |               | 09/30/20     | 07/30/20    | United Technologies Corporation                       | San Diego         |        21 | Closure Permanent            |
|  69 |               | 09/26/20     | 07/31/20    | Kixeye, Inc.                                          | San Francisco     |        62 | Layoff Permanent             |
|  70 |               | 10/02/20     | 07/31/20    | Boston Scientific Corporation                         | Fremont           |       284 | Closure Permanent            |
|  71 |               | 10/02/20     | 07/31/20    | Boston Scientific Corporation                         | San Jose          |       171 | Closure Permanent            |
|  72 |               | 09/30/20     | 08/03/20    | Ruan Transportation Corporation                       | Ripon             |        78 | Closure Permanent            |
|  73 |               | 09/30/20     | 08/03/20    | Armstrong Growers                                     | San Juan          |        56 | Closure Permanent            |
|  74 |               | 09/28/20     | 08/03/20    | CR Briggs Corporation                                 | Trona             |         5 | Layoff Permanent             |
|  75 |               | 08/03/20     | 08/03/20    | Molycorp Minerals LLC                                 | Mountain Pass     |       474 | Layoff Permanent             |
|  76 |               | 09/30/20     | 08/03/20    | Binder & Binder                                       | Orange            |        70 | Closure Permanent            |
|  77 |               | 09/30/20     | 08/03/20    | Driscoll's Strawberry Associates, Inc.                | Salinas           |        38 | Closure Permanent            |
|  78 |               | 10/23/20     | 08/04/20    | Hilton San Francisco Union Square                     | San Francisco     |        78 | Layoff Permanent             |
|  79 |               | 10/05/20     | 08/04/20    | Owens-Brockway Glass Container Inc.                   | Oakland           |       202 | Closure Permanent            |
|  80 |               | 07/30/20     | 08/05/20    | Abbott Vascular                                       | Temecula          |       133 | Layoff Permanent             |
|  81 |               | 07/30/20     | 08/05/20    | Abbott Vascular                                       | Temecula          |         3 | Layoff Permanent             |
|  82 |               | 07/30/20     | 08/05/20    | Abbott Vascular                                       | Temecula          |        27 | Layoff Permanent             |
|  83 |               | 07/30/20     | 08/05/20    | Abbott Vascular                                       | Santa Clara       |        81 | Layoff Permanent             |
|  84 |               | 09/11/20     | 08/07/20    | Anna's Linens, Inc.                                   | Costa Mesa        |        29 | Closure Permanent            |
|  85 |               | 10/02/20     | 08/10/20    | Boeing Company                                        | Long Beach        |         8 | Layoff Unknown at this time  |
|  86 |               | 10/09/20     | 08/10/20    | Safeway, Inc.                                         | Pleasanton        |        37 | Layoff Unknown at this time  |
|  87 |               | 10/05/20     | 08/11/20    | CGR/Thompson Industries, Inc.                         | Fullerton         |       102 | Closure Permanent            |
|  88 |               | 10/15/20     | 08/12/20    | BI NUTRACEUTICALS, INC.                               | Long Beach        |        54 | Layoff Permanent             |
|  89 |               | 09/30/20     | 08/13/20    | Lyris, Inc.                                           | Emeryville        |        72 | Layoff Permanent             |
|  90 |               | 09/30/20     | 08/13/20    | Lyris, Inc.                                           | Santa Clara       |         7 | Layoff Permanent             |
|  91 |               | 10/09/20     | 08/13/20    | Ralphs Grocery Company                                | Pico Rivera       |        67 | Layoff Permanent             |
|  92 |               | 10/12/20     | 08/13/20    | Chevron                                               | San Ramon         |       430 | Layoff Permanent             |
|  93 |               | 10/12/20     | 08/13/20    | Chevron                                               | Richmond          |        70 | Layoff Permanent             |
|  94 |               | 10/16/20     | 08/13/20    | Quest Diagnostics Incorporated                        | San Jose          |        78 | Layoff Permanent             |
|  95 |               | 10/23/20     | 08/13/20    | Motorola Mobility LLC                                 | Sunnyvale         |       213 | Layoff Permanent             |
|  96 |               | 10/14/20     | 08/14/20    | Raytheon Intelligence, Information &                  | Chula Vista       |        41 | Closure Unknown at this time |
|  97 |               | 10/13/20     | 08/17/20    | Haggen, Inc. (Store #2154)                            | Newbury Park      |        38 | Closure Permanent            |
|  98 |               | 10/13/20     | 08/17/20    | Haggen, Inc. (Store #2157)                            | Simi Valley       |        48 | Closure Permanent            |
|  99 |               | 10/13/20     | 08/17/20    | Haggen, Inc. (Store #2199)                            | Simi Valley       |        53 | Closure Permanent            |
| 100 |               | 10/13/20     | 08/17/20    | Haggen, Inc. (Store #2201)                            | Irvine            |        47 | Closure Permanent            |
| 101 |               | 10/13/20     | 08/17/20    | Haggen, Inc. (Store #2203)                            | Mission Viejo     |        41 | Closure Permanent            |
| 102 |               | 10/13/20     | 08/17/20    | Haggen, Inc. (Store #2213)                            | Santa Clarita     |        33 | Closure Permanent            |
| 103 |               | 10/13/20     | 08/17/20    | Haggen, Inc. (Store #2215)                            | Tustin            |        33 | Closure Permanent            |
| 104 |               | 10/13/20     | 08/17/20    | Haggen, Inc. (Store #2216)                            | Tustin            |        36 | Closure Permanent            |
| 105 |               | 10/16/20     | 08/18/20    | Sky Aerospace Products                                | Los Angeles       |        53 | Closure Permanent            |
| 106 |               | 10/16/20     | 08/18/20    | Ralphs Grocery Company                                | Hermosa Beach     |        74 | Layoff Permanent             |
| 107 |               | 10/13/20     | 08/18/20    | Haggen, Inc. (Store #2183)                            | El Cajon          |        49 | Closure Permanent            |
| 108 |               | 10/13/20     | 08/18/20    | Haggen, Inc. (Store #2185)                            | La Mesa           |        37 | Closure Permanent            |
| 109 |               | 10/13/20     | 08/18/20    | Haggen, Inc. (Store #2191)                            | San Marcos        |        36 | Closure Permanent            |
| 110 |               | 10/13/20     | 08/18/20    | Haggen, Inc. (Store #2193)                            | Chula Vista       |        37 | Closure Permanent            |
| 111 |               | 10/13/20     | 08/18/20    | Haggen, Inc. (Store #2194)                            | Chula Vista       |        31 | Closure Permanent            |
| 112 |               | 10/13/20     | 08/18/20    | Haggen, Inc. (Store #2207)                            | San Ysidro        |        41 | Closure Permanent            |
| 113 |               | 10/13/20     | 08/18/20    | Haggen, Inc. (Store #2208)                            | Bakersfield       |        36 | Closure Permanent            |
| 114 |               | 10/13/20     | 08/18/20    | Haggen, Inc. (Store #2210)                            | Los Osos          |        24 | Closure Permanent            |
| 115 |               | 10/25/20     | 08/18/20    | Ralphs Grocery Company                                | Santa Ana         |        75 | Layoff Permanent             |
| 116 |               | 10/27/20     | 08/18/20    | USS-POSCO Industries                                  | Pittsburg         |       661 | Layoff Temporary             |
| 117 |               | 10/30/20     | 08/19/20    | Zimmer Biomet Dental (Aston)                          | Carlsbad          |        97 | Closure Permanent            |
| 118 |               | 10/30/20     | 08/19/20    | Zimmer Biomet Dental (El Camino)                      | Carlsbad          |       172 | Closure Permanent            |
| 119 |               | 10/30/20     | 08/19/20    | Zimmer Biomet, Inc. (Darwin Ct)                       | Carlsbad          |        10 | Closure Permanent            |
| 120 |               | 10/18/20     | 08/19/20    | KND Development 53, L.L.C., d/b/a                     | Hawaiian Gardens  |        94 | Closure Permanent            |
| 121 |               | 10/16/20     | 08/21/20    | Soitec Solar Industries, LLC                          | San Diego         |       100 | Closure Permanent            |
| 122 |               | 10/19/20     | 08/21/20    | Safeway                                               | Walnut Creek      |        63 | Closure Permanent            |
| 123 |               | 10/22/20     | 08/21/20    | XOMA (US) LLC                                         | Berkeley          |        39 | Layoff Permanent             |
| 124 |               | 10/23/20     | 08/21/20    | Western Digital Technologies, Inc.                    | Mountain View     |        56 | Layoff Permanent             |
| 125 |               | 10/23/20     | 08/21/20    | Western Digital Technologies, Inc.                    | Irvine            |        81 | Layoff Permanent             |
| 126 |               | 10/25/20     | 08/24/20    | Silgan Containers                                     | Riverbank         |       180 | Layoff Temporary             |
| 127 |               | 10/23/20     | 08/25/20    | Boeing Company                                        | Huntington Beach  |        26 | Layoff Unknown at this time  |
| 128 |               | 10/23/20     | 08/25/20    | Boeing Company                                        | Long Beach        |       113 | Layoff Unknown at this time  |
| 129 |               | 10/23/20     | 08/25/20    | Boeing Company                                        | El Segundo        |        39 | Layoff Unknown at this time  |
| 130 |               | 10/31/20     | 08/25/20    | Mesa Beverage Co., Inc.                               | Santa Rosa        |       137 | Layoff Permanent             |
| 131 |               | 10/24/20     | 08/26/20    | Patina Group NewCo, LLC.                              | Los Angeles       |       317 | Layoff Unknown at this time  |
| 132 |               | 08/31/20     | 08/27/20    | VT West, Inc.                                         | La Mirada         |        68 | Layoff Permanent             |
| 133 |               | 10/26/20     | 08/28/20    | Intuit Inc.                                           | San Diego         |         9 | Layoff Permanent             |
| 134 |               | 10/26/20     | 08/28/20    | Intuit Inc.                                           | Menlo Park        |         8 | Layoff Permanent             |
| 135 |               | 10/26/20     | 08/28/20    | Intuit Inc.                                           | San Francisco     |         2 | Layoff Permanent             |
| 136 |               | 10/26/20     | 08/28/20    | Intuit Inc.                                           | Woodland Hills    |        46 | Layoff Permanent             |
| 137 |               | 10/26/20     | 08/28/20    | Intuit Inc.                                           | Mountain View     |        45 | Layoff Permanent             |
| 138 |               | 10/30/20     | 08/28/20    | Sheraton Operating Corporation                        | Palm Desert       |       124 | Layoff Permanent             |
| 139 |               | 11/07/20     | 08/31/20    | San Jose Airport Garden Hotel                         | San Jose          |       123 | Closure Permanent            |
| 140 |               | 10/30/20     | 08/31/20    | Safeway Inc.                                          | Pleasanton        |         5 | Layoff Unknown at this time  |
| 141 |               | 10/30/20     | 09/01/20    | Boeing Company                                        | Long Beach        |         2 | Layoff Unknown at this time  |
| 142 |               | 10/30/20     | 09/01/20    | Boeing Company                                        | Huntington Beach  |         6 | Layoff Unknown at this time  |
| 143 |               | 10/30/20     | 09/01/20    | Boeing Company                                        | El Segundo        |       106 | Layoff Unknown at this time  |
| 144 |               | 11/01/20     | 09/01/20    | Kor Hotel Management, LLC dba Viceroy  San Francisco  |                   |        47 | Closure Temporary            |
| 145 |               | 11/02/20     | 09/02/20    | M.E. Fox & Company, Inc.                              | San Jose          |       127 | Layoff Permanent             |
| 146 |               | 11/03/20     | 09/03/20    | Sheraton Operating Corporation                        | Dana Point        |       610 | Closure Temporary            |
| 147 |               | 10/23/20     | 09/04/20    | Park 'N Fly San Diego and Park 'N Fly,                | San Diego         |        71 | Layoff Permanent             |
| 148 |               | 11/06/20     | 09/04/20    | Acclarent, Inc.                                       | Menlo Park        |       114 | Layoff Unknown at this time  |
| 149 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation                                    | Torrance          |         9 | Layoff Unknown at this time  |
| 150 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation                                    | Santa Monica      |        12 | Layoff Unknown at this time  |
| 151 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation (700 Wilshire store)               | Los Angeles       |        10 | Layoff Unknown at this time  |
| 152 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation (Broadway Arcade                   | Los Angeles       |         9 | Layoff Unknown at this time  |
| 153 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation (City National                     | Los Angeles       |         4 | Layoff Unknown at this time  |
| 154 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation (Pacific Center                    | Los Angeles       |        11 | Layoff Unknown at this time  |
| 155 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation (Roosevelt store)                  | Los Angeles       |         9 | Layoff Unknown at this time  |
| 156 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation (Two California                    | Los Angeles       |        10 | Layoff Unknown at this time  |
| 157 |               | 10/30/20     | 09/04/20    | FAMIMA Corporation (Union Station                     | Los Angeles       |        20 | Layoff Unknown at this time  |
| 158 |               | 11/02/20     | 09/04/20    | MiaSole Hi-Tech Corp. (Santa Clara-                   | Santa Clara       |         1 | Layoff Permanent             |
| 159 |               | 11/02/20     | 09/04/20    | MiaSole Hi-Tech Corp. (Santa Clara-                   | Santa Clara       |        64 | Layoff Permanent             |
| 160 |               | 11/02/20     | 09/04/20    | MiaSole Hi-Tech Corp. (Sunnyvale-                     | Sunnyvale         |        14 | Layoff Permanent             |
| 161 |               | 08/25/20     | 09/08/20    | BAE SYSTEMS                                           | San Francisco     |       134 | Layoff Temporary             |
| 162 |               | 11/06/20     | 09/10/20    | Mentor Worldwide LLC                                  | Santa Barbara     |        81 | Layoff Unknown at this time  |
| 163 |               | 11/09/20     | 09/10/20    | Collecto, Inc. dba EOS CCA                            | Novato            |        45 | Closure Permanent            |
| 164 |               | 11/10/20     | 09/10/20    | Western Digital Technologies, Inc.                    | Irvine            |       143 | Layoff Permanent             |
| 165 |               | 11/01/20     | 09/11/20    | Zenith Education Group                                | Santa Ana         |        18 | Layoff Permanent             |
| 166 |               | 09/04/20     | 09/11/20    | Quiksilver, Inc.                                      | Huntington Beach  |        77 | Layoff Permanent             |
| 167 |               | 11/06/20     | 09/14/20    | Boeing Company                                        | Long Beach        |         2 | Layoff Unknown at this time  |
| 168 |               | 11/09/20     | 09/14/20    | Seagate US LLC                                        | Cupertino         |        62 | Layoff Permanent             |
| 169 |               | 09/14/20     | 09/14/20    | Cisco Systems, Inc.                                   | San Jose          |        20 | Layoff Permanent             |
| 170 |               | 09/29/20     | 09/14/20    | AT&T                                                  | San Ramon         |       219 | Layoff Permanent             |
| 171 |               | 07/20/20     | 09/16/20    | GREE International, Inc.                              | San Francisco     |       114 | Layoff Unknown at this time  |
| 172 |               | 11/14/20     | 09/16/20    | Brake Parts Inc.                                      | Chowchilla        |       107 | Layoff Permanent             |
| 173 |               | 09/17/20     | 09/16/20    | Halliburton Energy Services, Inc.                     | Bakersfield       |        52 | Layoff Permanent             |
| 174 |               | 09/30/20     | 09/17/20    | CareFusion (Pacific Mesa)                             | San Diego         |        19 | Layoff Permanent             |
| 175 |               | 09/30/20     | 09/17/20    | CareFusion (Torrey View)                              | San Diego         |        76 | Layoff Permanent             |
| 176 |               | 11/20/20     | 09/17/20    | Qualcomm Incorporated                                 | San Jose          |        10 | Layoff Permanent             |
| 177 |               | 11/20/20     | 09/17/20    | Qualcomm Incorporated                                 | San Jose          |        14 | Layoff Permanent             |
| 178 |               | 11/20/20     | 09/17/20    | Qualcomm Incorporated                                 | San Jose          |        36 | Layoff Permanent             |
| 179 |               | 11/20/20     | 09/17/20    | Qualcomm Incorporated                                 | Santa Clara       |        70 | Layoff Permanent             |
| 180 |               | 11/20/20     | 09/18/20    | AbilityFirst                                          | Woodland Hills    |       105 | Closure Permanent            |
| 181 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        33 | Layoff Permanent             |
| 182 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        17 | Layoff Permanent             |
| 183 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | Carlsbad          |         1 | Layoff Permanent             |
| 184 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |         9 | Layoff Permanent             |
| 185 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        30 | Layoff Permanent             |
| 186 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        40 | Layoff Permanent             |
| 187 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |       358 | Layoff Permanent             |
| 188 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |       158 | Layoff Permanent             |
| 189 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        35 | Layoff Permanent             |
| 190 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |         6 | Layoff Permanent             |
| 191 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |       227 | Layoff Permanent             |
| 192 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |         9 | Layoff Permanent             |
| 193 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        21 | Layoff Permanent             |
| 194 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        14 | Layoff Permanent             |
| 195 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |       208 | Layoff Permanent             |
| 196 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        65 | Layoff Permanent             |
| 197 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |        80 | Layoff Permanent             |
| 198 |               | 11/20/20     | 09/18/20    | Qualcomm Incorporated                                 | San Diego         |         3 | Layoff Permanent             |
| 199 |               | 11/13/20     | 09/21/20    | Boeing Company                                        | El Segundo        |         3 | Layoff Temporary             |
| 200 |               | 11/16/20     | 09/22/20    | Intuit Inc.                                           | Woodland Hills    |        12 | Layoff Permanent             |
| 201 |               | 11/20/20     | 09/22/20    | Ralphs Grocery Company                                | Glendora          |        73 | Layoff Permanent             |
| 202 |               | 11/16/20     | 09/22/20    | MCC BB Property LLC                                   | Santa Barbara     |        23 | Layoff Permanent             |
| 203 |               | 11/22/20     | 09/22/20    | TGI Friday's                                          | Valencia          |        62 | Closure Permanent            |
| 204 |               | 01/06/20     | 09/24/20    | Medtronic Distribution                                | Mira Loma         |        21 | Layoff Permanent             |
| 205 |               | 11/23/20     | 09/24/20    | Bridgepoint Education, Inc.                           | San Diego         |        19 | Layoff Permanent             |
| 206 |               | 11/20/20     | 09/28/20    | Boeing Company                                        | El Segundo        |        54 | Layoff Permanent             |
| 207 |               | 11/20/20     | 09/28/20    | Boeing Company                                        | Long Beach        |        37 | Layoff Permanent             |
| 208 |               | 11/20/20     | 09/28/20    | Boeing Company                                        | Huntington Beach  |        23 | Layoff Permanent             |
| 209 |               | 11/20/20     | 09/28/20    | Safeway Inc.                                          | Pleasanton        |        15 | Layoff Unknown at this time  |
| 210 |               | 11/27/20     | 09/28/20    | Brice Manufacturing Company, Inc.                     | Pacoima           |         9 | Layoff Permanent             |
| 211 |               | 11/27/20     | 09/28/20    | Safeway, Inc.                                         | Pleasanton        |         4 | Layoff Unknown at this time  |
| 212 |               | 09/24/20     | 09/29/20    | Space Systems/Loral, LLC                              | Palo Alto         |        90 | Layoff Permanent             |
| 213 |               | 11/23/20     | 09/29/20    | Flagship Food Group                                   | Chatsworth        |       123 | Closure Permanent            |
| 214 |               | 05/01/20     | 09/30/20    | Packaging Advantage Corporation DBA                   | Los Angeles       |       192 | Closure Permanent            |
| 215 |               | 11/30/20     | 09/30/20    | San Diego Sports, LLC                                 | San Diego         |       123 | Closure Permanent            |
| 216 |               | 11/30/20     | 09/30/20    | United Technologies Corporation                       | San Diego         |         7 | Closure Permanent            |
| 217 |               | 11/27/20     | 10/01/20    | MobiTV, Inc.                                          | Emeryville        |        54 | Layoff Permanent             |
| 218 |               | 11/30/20     | 10/01/20    | SunEdison, Inc.                                       | Belmont           |        51 | Layoff Permanent             |
| 219 |               | 11/30/20     | 10/01/20    | SunEdison, Inc.                                       | San Francisco     |        11 | Layoff Permanent             |
| 220 |               | 12/04/20     | 10/01/20    | Fireman's Fund Insurance Company                      | Novato            |        73 | Layoff Permanent             |
| 221 |               | 12/01/20     | 10/02/20    | Menzies Aviation                                      | Los Angeles       |        51 | Layoff Permanent             |
| 222 |               | 01/16/20     | 10/02/20    | Kohl's Department Stores, Inc.                        | Santa Clara       |       119 | Closure Permanent            |
| 223 |               | 10/02/20     | 10/02/20    | Aerotek, Inc.                                         | Oxnard            |        58 | Layoff Unknown at this time  |
| 224 |               | 11/30/20     | 10/05/20    | Kythera Biopharmaceuticals, Inc.                      | Westlake Village  |       117 | Layoff Permanent             |
| 225 |               | 11/05/20     | 10/05/20    | GMRI, Inc.                                            | Los Angeles       |        83 | Closure Permanent            |
| 226 |               | 12/04/20     | 10/05/20    | Svenhard's Swedish Bakery                             | Oakland           |       162 | Closure Permanent            |
| 227 |               | 12/05/20     | 10/06/20    | Marine Terminal Corporation                           | Los Angeles       |       136 | Closure Permanent            |
| 228 |               | 12/05/20     | 10/06/20    | Marine Terminal Corporation                           | Oakland           |        39 | Closure Permanent            |
| 229 |               | 12/07/20     | 10/08/20    | MullinTBG                                             | El Segundo        |        50 | Layoff Permanent             |
| 230 |               | 12/07/20     | 10/08/20    | MullinTBG                                             | Irvine            |         3 | Layoff Permanent             |
| 231 |               | 09/25/20     | 10/08/20    | Onyx Pharmaceuticals, Inc                             | South San         |       377 | Closure Permanent            |
| 232 |               | 12/05/20     | 10/09/20    | Ports America Shared Services, Inc.                   | Terminal Island   |         7 | Layoff Permanent             |
| 233 |               | 12/05/20     | 10/09/20    | Ports America Shared Services, Inc.                   | Oakland           |         4 | Layoff Permanent             |
| 234 |               | 11/24/20     | 10/09/20    | Triple B Corporation                                  | Irwindale         |       124 | Closure Permanent            |
| 235 |               | 12/06/20     | 10/12/20    | Apro Distribution, LLC                                | Gardena           |        61 | Layoff Permanent             |
| 236 |               | 12/07/20     | 10/12/20    | Southern California Edison Company                    | Irwindale         |        32 | Layoff Permanent             |
| 237 |               | 12/28/20     | 10/12/20    | Barclays Capital Inc.                                 | San Francisco     |        20 | Layoff Permanent             |
| 238 |               | 12/28/20     | 10/12/20    | Barclays Capital Inc.                                 | Los Angeles       |        15 | Layoff Permanent             |
| 239 |               | 12/07/20     | 10/13/20    | General Dynamics Advanced Information  Thousand Oaks  |                   |        55 | Closure Permanent            |
| 240 |               | 12/18/20     | 10/13/20    | WD Media, LLC                                         | San Jose          |         2 | Layoff Permanent             |
| 241 |               | 12/18/20     | 10/13/20    | Western Digital, LLC                                  | Fremont           |       109 | Layoff Permanent             |
| 242 |               | 10/14/20     | 10/14/20    | TTM Technologies, Inc.                                | Milpitas          |       175 | Closure Unknown at this time |
| 243 |               | 12/01/20     | 10/14/20    | Zenith Education Group                                | Santa Ana         |        48 | Layoff Permanent             |
| 244 |               | 12/11/20     | 10/14/20    | Rockwell Collins, Inc.                                | Poway             |        29 | Layoff Permanent             |
| 245 |               | 12/13/20     | 10/14/20    | Twitter, Inc.                                         | San Francisco     |       240 | Layoff Permanent             |
| 246 |               | 12/19/20     | 10/14/20    | GENCO                                                 | Eastvale          |        68 | Layoff Permanent             |
| 247 |               | 11/30/20     | 10/15/20    | Boeing Company                                        | Long Beach        |         1 | Layoff Unknown at this time  |
| 248 |               | 12/11/20     | 10/15/20    | Boeing Company                                        | Huntington Beach  |        76 | Layoff Unknown at this time  |
| 249 |               | 12/11/20     | 10/15/20    | Boeing Company                                        | Long Beach        |        36 | Layoff Unknown at this time  |
| 250 |               | 12/11/20     | 10/15/20    | Boeing Company                                        | El Segundo        |       109 | Layoff Unknown at this time  |
| 251 |               | 12/15/20     | 10/16/20    | Danmer Custom Shutters                                | Van Nuys          |        70 | Layoff Permanent             |
| 252 |               | 12/15/20     | 10/16/20    | Marvell Semiconductor, Inc.                           | Aliso Viejo       |         9 | Layoff Permanent             |
| 253 |               | 12/15/20     | 10/16/20    | Marvell Semiconductor, Inc.                           | Santa Clara       |        12 | Layoff Permanent             |
| 254 |               | 12/18/20     | 10/16/20    | Bridgepoint Education, Inc.                           | San Diego         |         3 | Layoff Permanent             |
| 255 |               | 12/18/20     | 10/16/20    | Bridgepoint Education, Inc.                           | San Diego         |         1 | Layoff Permanent             |
| 256 |               | 10/02/20     | 10/19/20    | Hewlett-Packard Company                               | Palo Alto         |        65 | Layoff Permanent             |
| 257 |               | 12/22/20     | 10/19/20    | PCM, Inc.                                             | Laguna Woods      |       968 | Layoff Permanent             |
| 258 |               | 12/31/20     | 10/20/20    | DD Traders, Inc.                                      | San Francisco     |        18 | Layoff Permanent             |
| 259 |               | 12/20/20     | 10/20/20    | Tra Vigne                                             | Saint Helena      |       136 | Closure Permanent            |
| 260 |               | 01/01/20     | 10/21/20    | City Winery Napa                                      | Napa              |        37 | Closure Permanent            |
| 261 |               | 12/21/20     | 10/21/20    | XOMA (US) LLC                                         | Berkeley          |        13 | Layoff Permanent             |
| 262 |               | 12/20/20     | 10/21/20    | Microsoft Corporation                                 | San Diego         |         4 | Closure Unknown at this time |
| 263 |               | 12/15/20     | 10/22/20    | Hersha Hospitality Management LP                      | Los Angeles       |        71 | Closure Temporary            |
| 264 |               | 10/23/20     | 10/22/20    | Halliburton Energy Services, Inc.                     | Bakersfield       |         6 | Layoff Permanent             |
| 265 |               | 10/23/20     | 10/22/20    | Fresh & Easy, LLC                                     | Torrance          |       106 | Closure Permanent            |
| 266 |               | 10/23/20     | 10/22/20    | Fresh & Easy, LLC Campus Kitchen                      | Riverside         |       307 | Closure Permanent            |
| 267 |               | 10/23/20     | 10/22/20    | Fresh & Easy, LLC Campus Produce                      | Riverside         |       336 | Closure Permanent            |
| 268 |               | 10/23/20     | 10/22/20    | Fresh & Easy, LLC Distribution Center                 | Riverside         |       163 | Closure Permanent            |
| 269 |               | 12/20/20     | 10/22/20    | Wells Fargo                                           | Santa Ana         |        65 | Closure Unknown at this time |
| 270 |               | 12/21/20     | 10/22/20    | Bridgepoint Education, Inc.                           | San Diego         |         4 | Layoff Permanent             |
| 271 |               | 12/26/20     | 10/26/20    | The Declan Suites                                     | San Diego         |        35 | Layoff Unknown at this time  |
| 272 |               | 10/21/20     | 10/27/20    | Aqua Lung America, Inc.                               | Calexico          |        29 | Closure Permanent            |
| 273 |               | 12/22/20     | 10/27/20    | Napa Valley Grille                                    | Los Angeles       |        86 | Closure Temporary            |
| 274 |               | 12/28/20     | 10/27/20    | ISOLA USA Corp.                                       | Elk Grove         |        72 | Closure Permanent            |
| 275 |               | 12/31/20     | 10/27/20    | Arena Pharmaceuticals, Inc.                           | San Diego         |        79 | Layoff Permanent             |
| 276 |               | 09/12/20     | 10/28/20    | Raley's                                               | Sacramento        |        60 | Closure Permanent            |
| 277 |               | 10/22/20     | 10/28/20    | Haas Automation, Inc.                                 | Oxnard            |         6 | Layoff Permanent             |
| 278 |               | 12/27/20     | 10/28/20    | Raley's                                               | Stockton          |        54 | Closure Permanent            |
| 279 |               | 01/04/20     | 10/28/20    | CoreLogic                                             | Redwood City      |        21 | Closure Permanent            |
| 280 |               | 12/28/20     | 10/28/20    | Brice Manufacturing Company, Inc.                     | Pacoima           |        33 | Layoff Permanent             |
| 281 |               | 03/31/20     | 10/29/20    | Vishay Siliconix                                      | Santa Clara       |       292 | Layoff Permanent             |
| 282 |               | 10/26/20     | 10/30/20    | LG NanoH2O                                            | El Segundo        |        48 | Layoff Permanent             |
| 283 |               | 10/26/20     | 10/30/20    | LG NanoH2O                                            | Hawthorne         |        35 | Layoff Permanent             |
| 284 |               | 10/20/20     | 10/30/20    | BAE SYSTEMS                                           | San Francisco     |       163 | Layoff Temporary             |
| 285 |               | 10/20/20     | 10/30/20    | BAE SYSTEMS                                           | San Francisco     |         2 | Layoff Unknown at this time  |
| 286 |               | 12/31/20     | 10/30/20    | Boston Scientific Corporation                         | San Jose          |        75 | Layoff Permanent             |
| 287 |               | 12/31/20     | 10/30/20    | Safeway, Inc.                                         | Pleasanton        |        23 | Layoff Unknown at this time  |
| 288 |               | 12/31/20     | 11/02/20    | Verizon                                               | Irvine            |       171 | Layoff Permanent             |
| 289 |               | 12/31/20     | 11/02/20    | CEVA Logistics                                        | Stockton          |        56 | Layoff Permanent             |
| 290 |               | 01/03/20     | 11/02/20    | Greystone Manor                                       | Los Angeles       |        61 | Closure Temporary            |
| 291 |               | 01/03/20     | 11/04/20    | Grill Concepts                                        | Studio City       |        49 | Closure Temporary            |
| 292 |               | 11/03/20     | 11/04/20    | Cisco Systems, Inc.                                   | San Jose          |       126 | Layoff Permanent             |
| 293 |               | 01/03/20     | 11/05/20    | Scott's Restaurants, LLC                              | Costa Mesa        |       106 | Closure Permanent            |
| 294 |               | 01/01/20     | 11/05/20    | Boeing Company                                        | Huntington Beach  |        35 | Layoff Unknown at this time  |
| 295 |               | 01/01/20     | 11/05/20    | Boeing Company                                        | Long Beach        |        13 | Layoff Unknown at this time  |
| 296 |               | 01/01/20     | 11/05/20    | Boeing Company                                        | El Segundo        |        17 | Layoff Unknown at this time  |
| 297 |               | 01/07/20     | 11/05/20    | JC Entertainment Lighting Services, Inc.              | Sun Valley        |       169 | Layoff Permanent             |
| 298 |               | 01/06/20     | 11/06/20    | Microsemi                                             | Garden Grove      |        57 | Layoff Permanent             |
| 299 |               | 12/31/20     | 11/09/20    | Boeing Company                                        | Long Beach        |         1 | Layoff Unknown at this time  |
| 300 |               | 11/24/20     | 11/09/20    | Penske Logistics                                      | South Gate        |         6 | Closure Permanent            |
| 301 |               | 11/24/20     | 11/09/20    | Penske Logistics                                      | South Gate        |        54 | Layoff Permanent             |
| 302 |               | 12/10/20     | 11/09/20    | Golden State Drilling, Inc.                           | Bakersfield       |       104 | Layoff Permanent             |
| 303 |               | 11/01/20     | 11/09/20    | GMRI, Inc. d/b/a Yard House                           | San Diego         |       110 | Closure Permanent            |
| 304 |               | 01/08/20     | 11/10/20    | Hyundai Capital America                               | Irvine            |        25 | Layoff Permanent             |
| 305 |               | 01/08/20     | 11/10/20    | Hyundai Capital America                               | Newport Beach     |        16 | Layoff Permanent             |
| 306 |               | 01/11/20     | 11/12/20    | SWAY Management LLC                                   | Oakland           |       102 | Layoff Permanent             |
| 307 |               | 01/06/20     | 11/12/20    | Forever 21 Retail, Inc.                               | San Luis Obispo   |        72 | Closure Permanent            |
| 308 |               | 01/05/20     | 11/16/20    | AT&T                                                  | Commerce          |        99 | Layoff Permanent             |
| 309 |               | 01/18/20     | 11/17/20    | Exel Inc.                                             | Ontario           |        92 | Closure Permanent            |
| 310 |               | 01/16/20     | 11/17/20    | Macy's Century City Store                             | Los Angeles       |       117 | Closure Permanent            |
| 311 |               | 01/17/20     | 11/18/20    | Wells Fargo                                           | San Bernardino    |       122 | Closure Unknown at this time |
| 312 |               | 09/30/20     | 11/19/20    | Plexus Corp.                                          | Fremont           |       204 | Closure Permanent            |
| 313 |               | 01/18/20     | 11/20/20    | AT&T                                                  | Oakland           |        92 | Layoff Permanent             |
| 314 |               | 01/15/20     | 11/23/20    | Kim Lighting                                          | City of Industry  |       160 | Closure Unknown at this time |
| 315 |               | 12/31/20     | 11/23/20    | Rdio, Inc.                                            | San Francisco     |       123 | Closure Permanent            |
| 316 |               | 01/20/20     | 11/23/20    | Construction Specialty Service, Inc. dba              | Bakersfield       |        61 | Layoff Unknown at this time  |
| 317 |               | 01/22/20     | 11/23/20    | Complete Genomics, Inc.                               | Mountain View     |       145 | Layoff Unknown at this time  |
| 318 |               | 01/22/20     | 11/23/20    | Complete Genomics, Inc.                               | Sunnyvale         |        13 | Layoff Unknown at this time  |
| 319 |               | 01/22/20     | 11/23/20    | Complete Genomics, Inc.                               | San Jose          |        15 | Layoff Unknown at this time  |
| 320 |               | 01/22/20     | 11/23/20    | Complete Genomics, Inc.                               | San Francisco     |         6 | Layoff Unknown at this time  |
| 321 |               | 01/22/20     | 11/23/20    | Safeway Inc.                                          | Pleasanton        |         7 | Layoff Unknown at this time  |
| 322 |               | 01/22/20     | 11/25/20    | Boeing Company                                        | Huntington Beach  |        28 | Layoff Unknown at this time  |
| 323 |               | 01/22/20     | 11/25/20    | Boeing Company                                        | Long Beach        |        10 | Layoff Unknown at this time  |
| 324 |               | 01/22/20     | 11/25/20    | Boeing Company                                        | El Segundo        |        28 | Layoff Unknown at this time  |
| 325 |               | 01/31/20     | 11/30/20    | Runaway Two Five Corporation                          | Los Angeles       |        73 | Closure Temporary            |
| 326 |               | 02/01/20     | 11/30/20    | Ceradyne, Inc.                                        | Costa Mesa        |        65 | Closure Permanent            |
| 327 |               | 02/29/20     | 11/30/20    | Bay Valley Foods                                      | City of Industry  |        62 | Closure Permanent            |
| 328 |               | 02/29/20     | 11/30/20    | YP Advertising & Publishing LLC                       | Pleasanton        |        68 | Layoff Permanent             |
| 329 |               | 12/07/20     | 12/01/20    | Brinderson, L.P.                                      | Tupman            |        55 | Closure Permanent            |
| 330 |               | 12/07/20     | 12/01/20    | Brinderson, L.P.                                      | Taft              |         9 | Closure Permanent            |
| 331 |               | 12/07/20     | 12/01/20    | Brinderson, L.P.                                      | Maricopa          |         2 | Closure Permanent            |
| 332 |               | 12/07/20     | 12/01/20    | Brinderson, L.P.                                      | Shafter           |        14 | Closure Permanent            |
| 333 |               | 12/07/20     | 12/01/20    | Brinderson, L.P.                                      | Lost Hills        |         8 | Closure Permanent            |
| 334 |               | 12/07/20     | 12/01/20    | Brinderson, L.P.                                      | Lost Hills        |        56 | Closure Permanent            |
| 335 |               | 12/07/20     | 12/01/20    |                                                       | Coalinga          |        84 | Closure Permanent            |
| 336 |               | 12/07/20     | 12/01/20    | Brinderson, L.P.                                      | San Ardo          |       101 | Closure Permanent            |
| 337 |               | 12/04/20     | 12/02/20    | Symantec Corporation                                  | Mountain View     |        44 | Layoff Permanent             |
| 338 |               | 01/31/20     | 12/02/20    | United Technologies Corporation                       | San Diego         |        31 | Closure Permanent            |
| 339 |               | 02/01/20     | 12/02/20    | CommScope                                             | San Jose          |        90 | Closure Permanent            |
| 340 |               | 01/24/20     | 12/04/20    | P.F. Chang's China Bistro, Inc.                       | Los Angeles       |        86 | Closure Permanent            |
| 341 |               | 02/05/20     | 12/04/20    | Schneider National Carriers, Inc.                     | Fontana           |        65 | Closure Permanent            |
| 342 |               | 02/05/20     | 12/04/20    | Schneider National Carriers, Inc.                     | South El Monte    |        10 | Layoff Unknown at this time  |
| 343 |               | 02/02/20     | 12/04/20    | Volcano Corporation                                   | Rancho Cordova    |        60 | Layoff Permanent             |
| 344 |               | 11/30/20     | 12/07/20    | Santa Fe Distributing, LLC                            | Elk Grove         |        21 | Closure Permanent            |
| 345 |               | 11/30/20     | 12/07/20    | Santa Fe Distributing, LLC                            | Hayward           |        21 | Closure Permanent            |
| 346 |               | 11/30/20     | 12/07/20    | Santa Fe Distributing, LLC                            | Walnut            |        21 | Closure Permanent            |
| 347 |               | 02/01/20     | 12/07/20    | Cedars-Sinai Medical Center                           | Los Angeles       |        57 | Layoff Permanent             |
| 348 |               | 02/01/20     | 12/07/20    | ITT Corporation                                       | Perris            |        18 | Closure Permanent            |
| 349 |               | 02/02/20     | 12/07/20    | C&J Well Services, Inc.                               | Bakersfield       |       161 | Layoff Permanent             |
| 350 |               | 02/03/20     | 12/07/20    | KVS Transportation, Inc.                              | Bakersfield       |       111 | Layoff Permanent             |
| 351 |               | 02/05/20     | 12/07/20    | The Wet Seal, LLC                                     | Foothill Ranch    |        20 | Layoff Permanent             |
| 352 |               | 02/08/20     | 12/07/20    | Marvell Semiconductor, Inc.                           | Santa Clara       |        13 | Layoff Permanent             |
| 353 |               | 02/13/20     | 12/07/20    | Sony Mobile Communications (USA) Inc.                 | San Mateo         |        80 | Layoff Permanent             |
| 354 |               | 12/07/20     | 12/08/20    | Abbott Vascular                                       | Redwood City      |       144 | Closure Permanent            |
| 355 |               | 02/09/20     | 12/09/20    | El Dorado Hotel and Kitchen                           | Sonoma            |       113 | Closure Temporary            |
| 356 |               | 02/08/20     | 12/09/20    | SunEdison, Inc.                                       | San Mateo         |        14 | Layoff Permanent             |
| 357 |               | 02/08/20     | 12/09/20    | SunEdison, Inc.                                       | Belmont           |         6 | Layoff Permanent             |
| 358 |               | 01/09/20     | 12/10/20    | abercrombie kids                                      | Milpitas          |        41 | Closure Permanent            |
| 359 |               | 01/18/20     | 12/10/20    | Citrix Systems, Inc.                                  | Santa Clara       |        54 | Layoff Permanent             |
| 360 |               | 12/07/20     | 12/10/20    | Anemostat, Inc.                                       | Carson            |        89 | Layoff Permanent             |
| 361 |               | 02/05/20     | 12/10/20    | Alta Resources                                        | Brea              |       121 | Layoff Permanent             |
| 362 |               | 02/08/20     | 12/10/20    | Bridgepoint Education, Inc.                           | San Diego         |         5 | Layoff Permanent             |
| 363 |               | 02/08/20     | 12/10/20    | Bridgepoint Education, Inc.                           | San Diego         |         1 | Layoff Permanent             |
| 364 |               | 01/11/20     | 12/11/20    | PPG Industries                                        | Fresno            |        45 | Closure Temporary            |
| 365 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Tulare            |        29 | Closure Permanent            |
| 366 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Ontario           |        32 | Closure Permanent            |
| 367 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Gilroy            |        25 | Closure Permanent            |
| 368 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Milpitas          |        18 | Closure Permanent            |
| 369 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Camarillo         |        16 | Closure Permanent            |
| 370 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Folsom            |        20 | Closure Permanent            |
| 371 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Vacaville         |        21 | Closure Permanent            |
| 372 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Fresno            |        22 | Layoff Permanent             |
| 373 |               | 04/01/20     | 12/11/20    | Suchman, LLC                                          | Modesto           |        21 | Closure Permanent            |
| 374 |               | 02/19/20     | 12/14/20    | Morrison Healthcare                                   | Chico             |        87 | Closure Permanent            |
| 375 |               | 02/06/20     | 12/14/20    | RSM Automotive, LLC, dba Santa                        | Rancho Santa      |        77 | Layoff Permanent             |
| 376 |               | 02/08/20     | 12/14/20    | MCC BB Property                                       | Santa Barbara     |        44 | Layoff Permanent             |
| 377 |               | 02/12/20     | 12/14/20    | Safeway, Inc.                                         | Pleasanton        |        33 | Layoff Unknown at this time  |
| 378 |               | 12/15/20     | 12/15/20    | Halliburton Energy Services, Inc.                     | Bakersfield       |        16 | Layoff Permanent             |
| 379 |               | 02/12/20     | 12/15/20    | Hudsloan Enterprises, Inc.                            | Los Angeles       |       265 | Closure Permanent            |
| 380 |               | 02/10/20     | 12/16/20    | Oakley, Inc.                                          | Foothill Ranch    |        76 | Layoff Unknown at this time  |
| 381 |               | 02/14/20     | 12/16/20    | Le Cordon Bleu North America, LLC                     | Pasadena          |       126 | Closure Permanent            |
| 382 |               | 02/15/20     | 12/17/20    | Parker Hannifin Corporation                           | Fontana           |        69 | Closure Unknown at this time |
| 383 |               | 12/14/20     | 12/18/20    | BAE SYSTEMS                                           | San Francisco     |         2 | Layoff Temporary             |
| 384 |               | 12/14/20     | 12/18/20    | BAE SYSTEMS                                           | San Francisco     |        61 | Layoff Temporary             |
| 385 |               | 02/25/20     | 12/18/20    | Gerawan Farming, Inc.                                 | Kerman            |      2551 | Closure Permanent            |
| 386 |               | 02/12/20     | 12/22/20    | Safeway, Inc.                                         | Pleasanton        |        33 | Layoff Unknown at this time  |
| 387 |               | 03/01/20     | 12/23/20    | Hyatt Regency Century Plaza                           | Los Angeles       |       796 | Closure Permanent            |
| 388 |               | 02/19/20     | 12/23/20    | Boeing Company                                        | Huntington Beach  |        35 | Layoff Unknown at this time  |
| 389 |               | 02/19/20     | 12/23/20    | Boeing Company                                        | Long Beach        |         9 | Layoff Unknown at this time  |
| 390 |               | 02/19/20     | 12/23/20    | Boeing Company                                        | El Segundo        |        35 | Layoff Unknown at this time  |
| 391 |               | 02/29/20     | 12/24/20    | TGI Friday's                                          | Carlsbad          |        45 | Closure Permanent            |
| 392 |               | 02/21/20     | 12/28/20    | Brake Parts Inc.                                      | Chowchilla        |        43 | Layoff Permanent             |
| 393 |               | 02/22/20     | 12/28/20    | AT&T                                                  | San Ramon         |       102 | Layoff Permanent             |
| 394 |               | 02/26/20     | 12/28/20    | Balda C. Brewer, Inc.                                 | Irvine            |       193 | Closure Permanent            |
| 395 |               | 02/29/20     | 12/28/20    | DNC Parks & Resorts at Yosemite, Inc.                 | Yosemite National |      1718 | Closure Permanent            |
| 396 |               | 02/29/20     | 12/30/20    | DuPont Displays, Inc.                                 | Santa Barbara     |        41 | Closure Permanent            |
| 397 |               | 03/04/20     | 12/30/20    | Parker Hannifin Corporation                           | Anaheim           |       197 | Closure Unknown at this time |
| 398 |               | 01/12/20     | 01/04/20    | Walnut Creek Pyramid Alehouse                         | Walnut Creek      |        49 | Closure Unknown at this time |
| 399 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | West Hollywood    |        25 | Closure Permanent            |
| 400 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | West Hollywood    |         5 | Closure Permanent            |
| 401 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | Los Angeles       |        28 | Closure Permanent            |
| 402 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | Malibu            |         9 | Closure Permanent            |
| 403 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | Santa Monica      |        45 | Closure Permanent            |
| 404 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | Newport Beach     |         9 | Closure Permanent            |
| 405 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | Brea              |         6 | Closure Permanent            |
| 406 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | San Diego         |         8 | Closure Permanent            |
| 407 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | San Diego         |        13 | Closure Permanent            |
| 408 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | Santa Barbara     |         9 | Closure Permanent            |
| 409 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | Camarillo         |        13 | Closure Permanent            |
| 410 |               | 01/04/20     | 01/05/20    | A-List, Inc. & H-List Inc. dba Kitson                 | Arcadia           |        18 | Closure Permanent            |
| 411 |               | 03/09/20     | 01/07/20    | Adecco Group NA                                       | Palo Alto         |       108 | Closure Permanent            |
| 412 |               | 03/14/20     | 01/07/20    | Macy's Country Club Plaza Store                       | Sacramento        |       111 | Closure Permanent            |
| 413 |               | 03/14/20     | 01/07/20    | Macy's Irvine Spectrum Store                          | Irvine            |       112 | Closure Permanent            |
| 414 |               | 03/05/20     | 01/11/20    | Thomson Licensing LLC                                 | Burbank           |         7 | Closure Unknown at this time |
| 415 |               | 03/07/20     | 01/11/20    | AT&T                                                  | San Ramon         |         1 | Layoff Permanent             |
| 416 |               | 03/07/20     | 01/11/20    | Pacific Bell Telephone Company                        | San Ramon         |         1 | Layoff Permanent             |
| 417 |               | 03/11/20     | 01/12/20    | Safeway Inc.                                          | Pleasanton        |         7 | Layoff Unknown at this time  |
| 418 |               | 02/29/20     | 01/12/20    | Molina Hospital Management, Inc.                      | Long Beach        |       385 | Layoff Permanent             |
| 419 |               | 03/31/20     | 01/13/20    | Zodiac Pool Systems, Inc.                             | Vista             |        52 | Closure Permanent            |
| 420 |               | 01/15/20     | 01/13/20    | GoPro, Inc.                                           | San Mateo         |        77 | Layoff Permanent             |
| 421 |               | 03/15/20     | 01/14/20    | Qualcomm Incorporated                                 | San Diego         |        32 | Layoff Unknown at this time  |
| 422 |               | 03/18/20     | 01/14/20    | ConAgra Foods, Inc.                                   | Helm              |       102 | Closure Permanent            |
| 423 |               | 04/29/20     | 01/14/20    | El Dorado Berry Farms, LLC (Adam                      | Santa Maria       |       218 | Layoff Permanent             |
| 424 |               | 04/29/20     | 01/14/20    | El Dorado Berry Farms, LLC (Bognuda                   | Santa Maria       |       180 | Layoff Permanent             |
| 425 |               | 04/29/20     | 01/14/20    | El Dorado Berry Farms, LLC (Coyote                    | Santa Maria       |       123 | Layoff Permanent             |
| 426 |               | 04/29/20     | 01/14/20    | El Dorado Berry Farms, LLC (Monighetti                | Santa Maria       |        30 | Layoff Permanent             |
| 427 |               | 04/29/20     | 01/14/20    | El Dorado Berry Farms, LLC (Punta de                  | Santa Maria       |       103 | Layoff Permanent             |
| 428 |               | 06/24/20     | 01/14/20    | Superior Farming, LLC                                 | Santa Maria       |       225 | Layoff Permanent             |
| 429 |               | 03/01/20     | 01/15/20    | HD Supply Management, Inc.                            | San Diego         |        97 | Layoff Permanent             |
| 430 |               | 04/15/20     | 01/15/20    | Walmart                                               | Oakland           |       397 | Closure Permanent            |
| 431 |               | 04/15/20     | 01/15/20    | Walmart                                               | San Bernardino    |        89 | Closure Permanent            |
| 432 |               | 04/15/20     | 01/15/20    | Walmart                                               | Hawaiian Gardens  |        77 | Closure Permanent            |
| 433 |               | 04/15/20     | 01/15/20    | Walmart                                               | Bell Gardens      |        85 | Closure Permanent            |
| 434 |               | 04/15/20     | 01/15/20    | Walmart                                               | Long Beach        |       295 | Closure Permanent            |
| 435 |               | 04/15/20     | 01/15/20    | Walmart                                               | Altadena          |        79 | Closure Permanent            |
| 436 |               | 04/15/20     | 01/15/20    | Walmart                                               | San Jose          |       210 | Closure Permanent            |
| 437 |               | 04/15/20     | 01/15/20    | Walmart                                               | Los Angeles       |       324 | Closure Permanent            |
| 438 |               | 04/15/20     | 01/15/20    | Walmart                                               | Los Angeles       |        96 | Closure Permanent            |
| 439 |               | 07/10/20     | 01/15/20    | MAC Berry Farms, LLC                                  | Camarillo         |        90 | Layoff Permanent             |
| 440 |               | 04/10/20     | 01/19/20    | Sears, Roebuck and Co.                                | San Mateo         |       110 | Closure Permanent            |
| 441 |               | 04/30/20     | 01/19/20    | Al Jazeera America, LLC                               | San Francisco     |        25 | Closure Permanent            |
| 442 |               | 04/30/20     | 01/19/20    | Al Jazeera America, LLC                               | Culver City       |        19 | Closure Permanent            |
| 443 |               | 04/30/20     | 01/19/20    | Al Jazeera America, LLC                               | Los Angeles       |         6 | Closure Permanent            |
| 444 |               | 03/15/20     | 01/19/20    | Brice Manufacturing Company, Inc.                     | Pacoima           |         1 | Layoff Permanent             |
| 445 |               | 03/21/20     | 01/19/20    | Georgia-Pacific Corrugated LLC                        | Buena Park        |       148 | Closure Permanent            |
| 446 |               | 03/07/20     | 01/20/20    | Citrix Systems, Inc.                                  | Santa Clara       |        65 | Layoff Permanent             |
| 447 |               | 03/19/20     | 01/20/20    | Outer Harbor Terminal LLC                             | Oakland           |        59 | Closure Permanent            |
| 448 |               | 03/20/20     | 01/20/20    | Destination Geary Street Management                   | San Francisco     |        58 | Layoff Permanent             |
| 449 |               | 03/20/20     | 01/20/20    | Microsoft Corporation                                 | San Diego         |         2 | Closure Permanent            |
| 450 |               | 01/20/20     | 01/21/20    | Cisco Systems, Inc.                                   | San Jose          |       123 | Layoff Permanent             |
| 451 |               | 03/18/20     | 01/22/20    | Coastal Green Vegetable Company, LLC                  | Oxnard            |        88 | Closure Permanent            |
| 452 |               | 03/21/20     | 01/22/20    | Pathology, Inc.                                       | Torrance          |       388 | Layoff Permanent             |
| 453 |               | 12/23/20     | 01/22/20    | Highland Metals Inc. and EZFuture Inc.                | San Jose          |        17 | Closure Permanent            |
| 454 |               | 01/21/20     | 01/22/20    | Centrally Grown, Inc.                                 | Cambria           |        16 | Closure Unknown at this time |
| 455 |               | 03/19/20     | 01/25/20    | F&E Aircraft Maintenance DBA FEAM                     | Los Angeles       |        25 | Layoff Permanent             |
| 456 |               | 01/20/20     | 01/25/20    | Sage North America                                    | Irvine            |        67 | Layoff Permanent             |
| 457 |               | 03/25/20     | 01/25/20    | Allianz Global Risks US Insurance                     | Petaluma          |        38 | Layoff Permanent             |
| 458 |               | 03/31/20     | 01/25/20    | EXP Pharmaceutical Services Corp.                     | Fremont           |       111 | Closure Permanent            |
| 459 |               | 03/13/20     | 01/26/20    | Kmart                                                 | Anaheim           |       107 | Closure Permanent            |
| 460 |               | 03/13/20     | 01/26/20    | Kmart                                                 | Chula Vista       |       103 | Closure Permanent            |
| 461 |               | 04/03/20     | 01/26/20    | Kmart                                                 | Citrus Heights    |        86 | Closure Permanent            |
| 462 |               | 03/25/20     | 01/26/20    | Boeing Company                                        | Huntington Beach  |        15 | Layoff Unknown at this time  |
| 463 |               | 03/25/20     | 01/26/20    | Boeing Company                                        | Long Beach        |        11 | Layoff Unknown at this time  |
| 464 |               | 03/25/20     | 01/26/20    | Boeing Company                                        | El Segundo        |         6 | Layoff Unknown at this time  |
| 465 |               | 03/25/20     | 01/26/20    | Sierra Pacific Industries                             | Arcata            |       125 | Closure Permanent            |
| 466 |               | 01/26/20     | 01/26/20    | Pearson Education                                     | San Francisco     |        34 | Layoff Unknown at this time  |
| 467 |               | 03/28/20     | 01/26/20    | Ditech Financial LLC.                                 | Costa Mesa        |        87 | Closure Permanent            |
| 468 |               | 03/28/20     | 01/26/20    | SPX FLOW, Inc.                                        | Modesto           |         8 | Closure Permanent            |
| 469 |               | 03/28/20     | 01/26/20    | VMware, Inc.                                          | Palo Alto         |       170 | Layoff Permanent             |
| 470 |               | 03/29/20     | 01/27/20    | VSE Corporation                                       | Jolon             |        70 | Layoff Permanent             |
| 471 |               | 03/29/20     | 01/27/20    | VSE Corporation                                       | Barstow           |        13 | Layoff Permanent             |
| 472 |               | 03/29/20     | 01/27/20    | VSE Corporation                                       | Barstow           |        27 | Layoff Permanent             |
| 473 |               | 03/28/20     | 01/28/20    | Eaton's Cooper Lighting                               | Richmond          |        47 | Closure Permanent            |
| 474 |               | 03/29/20     | 01/28/20    | VSE Corporation                                       | Mountain View     |         2 | Layoff Permanent             |
| 475 |               | 03/29/20     | 01/28/20    | VSE Corporation                                       | Riverside         |         1 | Layoff Permanent             |
| 476 |               | 03/29/20     | 01/28/20    | VSE Corporation                                       | Los Alamitos      |         1 | Layoff Permanent             |
| 477 |               | 03/29/20     | 01/28/20    | VSE Corporation                                       | Van Nuys          |         1 | Layoff Permanent             |
| 478 |               | 04/01/20     | 01/29/20    | Rockwell Collins, Inc.                                | Poway             |         2 | Layoff Unknown at this time  |
| 479 |               | 03/29/20     | 01/29/20    | Hub City Terminals, Inc.                              | Brea              |        12 | Closure Permanent            |
| 480 |               | 03/31/20     | 01/29/20    | Hub Group Trucking, Inc.                              | Ontario           |       131 | Closure Permanent            |
| 481 |               | 03/31/20     | 01/29/20    | Toshiba America Electronic                            | Livermore         |        50 | Closure Permanent            |
| 482 |               | 03/31/20     | 01/29/20    | Urban Fulfillment Services, LLC                       | Westlake Village  |       192 | Closure Permanent            |
| 483 |               | 03/31/20     | 02/01/20    | Cardinal Health                                       | Elk Grove         |        58 | Layoff Permanent             |
| 484 |               | 03/31/20     | 02/01/20    | Cardinal Health                                       | Valencia          |        15 | Layoff Permanent             |
| 485 |               | 04/01/20     | 02/01/20    | Westin St. Francis                                    | San Francisco     |        52 | Closure Permanent            |
| 486 |               | 04/04/20     | 02/01/20    | Practice Fusion, Inc.                                 | San Francisco     |        74 | Layoff Permanent             |
| 487 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | Sacramento        |         3 | Layoff Permanent             |
| 488 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | El Segundo        |         9 | Layoff Permanent             |
| 489 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | Ontario           |         2 | Layoff Permanent             |
| 490 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | Walnut Creek      |         1 | Layoff Permanent             |
| 491 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | Lodi              |        99 | Layoff Permanent             |
| 492 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | San Francisco     |        78 | Layoff Permanent             |
| 493 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | Rancho Cordova    |        55 | Layoff Permanent             |
| 494 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | Woodland Hills    |        60 | Layoff Permanent             |
| 495 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | Costa Mesa        |         2 | Layoff Permanent             |
| 496 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | San Jose          |         1 | Layoff Permanent             |
| 497 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | El Dorado Hills   |       137 | Layoff Permanent             |
| 498 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | San Diego         |         3 | Layoff Permanent             |
| 499 |               | 03/28/20     | 02/02/20    | Blue Shield of California                             | Redding           |        10 | Layoff Permanent             |
| 500 |               | 03/31/20     | 02/02/20    | Broadcom and Avago                                    | Irvine            |       687 | Layoff Permanent             |
| 501 |               | 03/31/20     | 02/02/20    | Broadcom and Avago                                    | Irvine            |         2 | Layoff Permanent             |
| 502 |               | 03/31/20     | 02/02/20    | Broadcom and Avago                                    | San Jose          |        33 | Layoff Permanent             |
| 503 |               | 03/31/20     | 02/02/20    | Broadcom and Avago freedom                            | Santa Clara       |        63 | Layoff Permanent             |
| 504 |               | 03/31/20     | 02/02/20    | Broadcom and Avago lafayette                          | Santa Clara       |         1 | Layoff Permanent             |
| 505 |               | 03/31/20     | 02/02/20    | Broadcom and Avago mission                            | Santa Clara       |        57 | Layoff Permanent             |
| 506 |               | 03/31/20     | 02/02/20    | Broadcom and Avago sunnyvale                          | Sunnyvale         |        26 | Layoff Permanent             |
| 507 |               | 04/01/20     | 02/02/20    | Moog Inc.                                             | Milpitas          |        22 | Layoff Permanent             |
| 508 |               | 04/01/20     | 02/03/20    | Boeing Company                                        | Huntington Beach  |        15 | Layoff Unknown at this time  |
| 509 |               | 04/01/20     | 02/03/20    | Boeing Company                                        | El Segundo        |        25 | Layoff Unknown at this time  |
| 510 |               | 03/31/20     | 02/03/20    | United Technologies Corporation                       | San Diego         |       103 | Closure Permanent            |
| 511 |               | 04/03/20     | 02/03/20    | Bon Appetit Management Company                        | San Francisco     |        79 | Closure Permanent            |
| 512 |               | 04/01/20     | 02/03/20    | Forever 21 Retail, Inc.                               | Hanford           |        51 | Closure Permanent            |
| 513 |               | 04/01/20     | 02/03/20    | Forever 21 Retail, Inc.                               | Yuba City         |        56 | Closure Permanent            |
| 514 |               | 04/05/20     | 02/03/20    | Freeport-McMoRan Oil & Gas LLC                        | Orcutt            |        64 | Layoff Permanent             |
| 515 |               | 02/19/20     | 02/04/20    | VF Contemporary Brands, Inc.                          | Vernon            |        73 | Closure Permanent            |
| 516 |               | 04/10/20     | 02/05/20    | Autodesk, Inc.                                        | San Francisco     |        60 | Layoff Permanent             |
| 517 |               | 04/10/20     | 02/05/20    | Autodesk, Inc.                                        | San Francisco     |         8 | Layoff Permanent             |
| 518 |               | 04/10/20     | 02/05/20    | Autodesk, Inc.                                        | San Francisco     |         3 | Layoff Permanent             |
| 519 |               | 04/10/20     | 02/05/20    | Autodesk, Inc.                                        | San Rafael        |        48 | Layoff Permanent             |
| 520 |               | 04/04/20     | 02/08/20    | Visiting Nurse Association of the Inland              | Victorville       |       178 | Closure Permanent            |
| 521 |               | 04/08/20     | 02/08/20    | Albertsons Companies                                  | Pleasanton        |        24 | Layoff Unknown at this time  |
| 522 |               | 01/28/20     | 02/09/20    | Space Systems/Loral, LLC                              | Palo Alto         |       122 | Layoff Permanent             |
| 523 |               | 02/04/20     | 02/09/20    | B&H Education Holdings, LLC & B&H                     | Whittier          |        73 | Closure Permanent            |
| 524 |               | 04/09/20     | 02/09/20    | JCPenney                                              | Cupertino         |       147 | Closure Permanent            |
| 525 |               | 04/08/20     | 02/09/20    | Inland Early Steps Services, Inc.                     | Corona            |        79 | Closure Permanent            |
| 526 |               | 04/01/20     | 02/10/20    |                                                       | Long Beach        |        64 | Closure Permanent            |
| 527 |               | 04/11/20     | 02/10/20    | Yahoo! Inc.                                           | Sunnyvale         |       107 | Layoff Permanent             |
| 528 |               | 04/08/20     | 02/12/20    | Southern California Edison Company                    | Rosemead          |        48 | Layoff Permanent             |
| 529 |               | 04/15/20     | 02/12/20    | Kraft Heinz Foods Company                             | San Leandro       |       117 | Closure Permanent            |
| 530 |               | 06/15/20     | 02/12/20    | Pinecrest Schools                                     | Woodland Hills    |        71 | Closure Permanent            |
| 531 |               | 06/15/20     | 02/12/20    | Pinecrest Schools                                     | Canyon Country    |        24 | Closure Permanent            |
| 532 |               | 06/15/20     | 02/12/20    | Pinecrest Schools                                     | Moorpark          |        37 | Closure Permanent            |
| 533 |               | 06/15/20     | 02/12/20    | Pinecrest Schools                                     | Thousand Oaks     |        37 | Closure Permanent            |
| 534 |               | 06/15/20     | 02/12/20    | Pinecrest Schools                                     | Simi Valley       |        44 | Closure Permanent            |
| 535 |               | 01/22/20     | 02/16/20    | HP Inc.                                               | Palo Alto         |        74 | Layoff Permanent             |
| 536 |               | 02/10/20     | 02/16/20    | Symantec Corporation                                  | Mountain View     |         6 | Layoff Permanent             |
| 537 |               | 04/17/20     | 02/16/20    | The Ritz-Carlton, Lake Tahoe                          | Truckee           |       253 | Closure Temporary            |
| 538 |               | 02/18/20     | 02/16/20    | Toshiba America Information Systems,                  | Irvine            |         7 | Layoff Permanent             |
| 539 |               | 04/18/20     | 02/16/20    | SK Textile, Inc.                                      | Vernon            |        51 | Closure Permanent            |
| 540 |               | 04/18/20     | 02/16/20    | Odwalla Inc.                                          | Dinuba            |       164 | Closure Permanent            |
| 541 |               | 04/12/20     | 02/17/20    | TangoMe, Inc.                                         | Mountain View     |        53 | Layoff Permanent             |
| 542 |               | 04/18/20     | 02/17/20    | Yahoo! Inc.                                           | Sunnyvale         |       127 | Layoff Permanent             |
| 543 |               | 04/18/20     | 02/17/20    | Yahoo! Inc.                                           | San Francisco     |        45 | Layoff Permanent             |
| 544 |               | 04/18/20     | 02/17/20    | Yahoo! Inc.                                           | Los Angeles       |        60 | Layoff Permanent             |
| 545 |               | 04/18/20     | 02/17/20    | Yahoo! Inc.                                           | Burbank           |        90 | Closure Permanent            |
| 546 |               | 02/26/20     | 02/18/20    | Valley Fine Foods                                     | Benicia           |        94 | Closure Permanent            |
| 547 |               | 04/17/20     | 02/18/20    | California Resources Corporation                      | Bakersfield       |        74 | Layoff Permanent             |
| 548 |               | 04/18/20     | 02/18/20    | Pacific Harvest, Inc.                                 | Guadalupe         |        75 | Layoff Permanent             |
| 549 |               | 02/09/20     | 02/22/20    | BAE SYSTEMS                                           | San Francisco     |         2 | Layoff Temporary             |
| 550 |               | 02/09/20     | 02/22/20    | BAE SYSTEMS                                           | San Francisco     |       228 | Layoff Temporary             |
| 551 |               | 04/19/20     | 02/22/20    | Brake Parts Inc.                                      | Chowchilla        |       111 | Layoff Permanent             |
| 552 |               | 04/19/20     | 02/22/20    | Vintage Estates                                       | Yountville        |       326 | Layoff Permanent             |
| 553 |               | 04/27/20     | 02/22/20    | BCBG Max Azria Group, LLC                             | Vernon            |        71 | Layoff Permanent             |
| 554 |               | 07/31/20     | 02/23/20    | Mallinckrodt Pharmaceuticals                          | Hayward           |        43 | Closure Permanent            |
| 555 |               | 03/31/20     | 02/23/20    | Kabam LA                                              | Los Angeles       |         1 | Layoff Permanent             |
| 556 |               | 03/31/20     | 02/23/20    | Kabam, Inc.                                           | San Francisco     |        75 | Layoff Permanent             |
| 557 |               | 04/25/20     | 02/23/20    | TGI Friday's                                          | Laguna Niguel     |        51 | Closure Permanent            |
| 558 |               | 03/31/20     | 02/24/20    | Freedom Communications, Inc.                          | Santa Ana         |       969 | Layoff Unknown at this time  |
| 559 |               | 04/18/20     | 02/24/20    | Maslow Media Group, Inc.                              | San Francisco     |        17 | Layoff Permanent             |
| 560 |               | 04/18/20     | 02/24/20    | Maslow Media Group, Inc.                              | Culver City       |         2 | Layoff Permanent             |
| 561 |               | 04/18/20     | 02/24/20    | Maslow Media Group, Inc.                              | Los Angeles       |        19 | Layoff Permanent             |
| 562 |               | 04/22/20     | 02/24/20    | Boeing Company                                        | Huntington Beach  |        44 | Layoff Unknown at this time  |
| 563 |               | 04/22/20     | 02/24/20    | Boeing Company                                        | Long Beach        |         3 | Layoff Unknown at this time  |
| 564 |               | 04/22/20     | 02/24/20    | Boeing Company                                        | El Segundo        |        45 | Layoff Unknown at this time  |
| 565 |               | 04/01/20     | 02/25/20    | Imagine Communications Corp.                          | Sunnyvale         |        50 | Closure Permanent            |
| 566 |               | 04/25/20     | 02/25/20    | Sutter Central Valley Hospitals dba                   | Modesto           |        83 | Layoff Unknown at this time  |
| 567 |               | 04/25/20     | 02/25/20    | Sutter Central Valley Hospitals dba                   | Modesto           |         7 | Layoff Unknown at this time  |
| 568 |               | 04/25/20     | 02/25/20    | Sutter Central Valley Hospitals dba                   | Modesto           |         1 | Layoff Unknown at this time  |
| 569 |               | 04/25/20     | 02/25/20    | Sutter Central Valley Hospitals dba                   | Modesto           |         1 | Layoff Unknown at this time  |
| 570 |               | 04/25/20     | 02/25/20    | Sutter Central Valley Hospitals dba                   | Modesto           |         4 | Layoff Unknown at this time  |
| 571 |               | 04/25/20     | 02/26/20    | Xerox Business Services, LLC                          | Bakersfield       |        15 | Layoff Permanent             |
| 572 |               | 05/04/20     | 02/29/20    | Pacific Bell Telephone Company                        | Tustin            |       121 | Layoff Permanent             |
| 573 |               | 02/29/20     | 02/29/20    | YourPeople, Inc. d/b/a Zenefits FTW                   | San Francisco     |        83 | Layoff Permanent             |
| 574 |               | 04/29/20     | 02/29/20    | Direct Energy                                         | Corona            |        47 | Closure Unknown at this time |
| 575 |               | 04/29/20     | 03/01/20    | Investor's Business Daily, Inc.                       | Los Angeles       |        42 | Layoff Permanent             |
| 576 |               | 04/29/20     | 03/01/20    | Investor's Business Daily, Inc.                       | Woodland Hills    |         1 | Layoff Permanent             |
| 577 |               | 04/29/20     | 03/01/20    | Investor's Business Daily, Inc.                       | Sunnyvale         |         3 | Layoff Permanent             |
| 578 |               | 04/29/20     | 03/01/20    | Investor's Business Daily, Inc.                       | San Diego         |         1 | Layoff Permanent             |
| 579 |               | 04/29/20     | 03/01/20    | Investor's Business Daily, Inc.                       | Coto De Caza      |         1 | Layoff Permanent             |
| 580 |               | 04/29/20     | 03/01/20    | Investor's Business Daily, Inc.                       | Valencia          |         1 | Layoff Permanent             |
| 581 |               | 03/01/20     | 03/01/20    | TiVo Inc.                                             | San Jose          |        52 | Layoff Permanent             |
| 582 |               | 03/01/20     | 03/01/20    | SurveyMonkey Inc.                                     | Palo Alto         |        57 | Layoff Permanent             |
| 583 |               | 04/30/20     | 03/01/20    | National Oilwell Varco                                | Orange            |       232 | Layoff Permanent             |
| 584 |               | 05/01/20     | 03/02/20    | Baxalta U.S. Inc.                                     | Thousand Oaks     |       100 | Layoff Permanent             |
| 585 |               | 05/01/20     | 03/02/20    | Baxalta U.S. Inc.                                     | Los Angeles       |       132 | Layoff Permanent             |
| 586 |               | 05/01/20     | 03/02/20    | Baxalta U.S. Inc.                                     | Van Nuys          |         7 | Layoff Permanent             |
| 587 |               | 04/30/20     | 03/03/20    | NetApp, Inc.                                          | Sunnyvale         |       376 | Layoff Permanent             |
| 588 |               | 05/03/20     | 03/04/20    | Southern Home Care Services, Inc.                     | San Leandro       |       123 | Closure Permanent            |
| 589 |               | 05/06/20     | 03/07/20    | Barnes & Noble College Booksellers, LLC Mountain View |                   |        40 | Closure Permanent            |
| 590 |               | 04/15/20     | 03/08/20    | Circor Inc.                                           | Corona            |        64 | Closure Permanent            |
| 591 |               | 05/04/20     | 03/08/20    | Circor Inc.                                           | Palm Springs      |        52 | Closure Permanent            |
| 592 |               | 05/13/20     | 03/08/20    | City National Bank                                    | Los Angeles       |         5 | Closure Permanent            |
| 593 |               | 05/31/20     | 03/08/20    | Memorial Health Services                              | San Clemente      |       194 | Closure Permanent            |
| 594 |               | 03/11/20     | 03/09/20    | Lyra Sound Inc.                                       | Cypress           |         4 | Closure Temporary            |
| 595 |               | 11/11/20     | 03/10/20    | Corning Incorporated                                  | Union City        |       193 | Closure Permanent            |
| 596 |               | 05/06/20     | 03/11/20    | Pacific Bell Telephone Company                        | Tustin            |         3 | Layoff Permanent             |
| 597 |               | 05/13/20     | 03/11/20    | Carbine, LLC                                          | Aliso Viejo       |        18 | Layoff Unknown at this time  |
| 598 |               | 03/11/20     | 03/14/20    | NC Interactive, LLC                                   | Aliso Viejo       |        11 | Layoff Permanent             |
| 599 |               | 04/10/20     | 03/14/20    | GMRI, Inc. dba Seasons 52                             | Santa Monica      |        70 | Closure Permanent            |
| 600 |               | 05/15/50     | 03/15/20    | Georgia-Pacific Gypsum LLC                            | San Leandro       |        65 | Closure Permanent            |
| 601 |               | 05/27/20     | 03/17/20    | Yamashiro, Inc.                                       | Los Angeles       |        88 | Closure Permanent            |
| 602 |               | 01/01/20     | 03/18/20    | Alma Farms, LLC                                       | Camarillo         |        73 | Closure Permanent            |
| 603 |               | 03/31/20     | 03/18/20    | McKesson Corporation                                  | San Francisco     |       158 | Layoff Permanent             |
| 604 |               | 05/17/20     | 03/21/20    | Williams-Sonoma, Inc.                                 | San Francisco     |        17 | Layoff Permanent             |
| 605 |               | 05/17/20     | 03/21/20    | Williams-Sonoma, Inc.                                 | San Francisco     |        30 | Layoff Permanent             |
| 606 |               | 05/17/20     | 03/21/20    | Williams-Sonoma, Inc.                                 | San Francisco     |        22 | Layoff Permanent             |
| 607 |               | 05/17/20     | 03/21/20    | Williams-Sonoma, Inc.                                 | San Francisco     |         5 | Layoff Permanent             |
| 608 |               | 05/17/20     | 03/21/20    | Williams-Sonoma, Inc.                                 | Richmond          |         1 | Layoff Permanent             |
| 609 |               | 05/17/20     | 03/21/20    | Williams-Sonoma, Inc.                                 | Brisbane          |         3 | Layoff Permanent             |
| 610 |               | 05/18/20     | 03/21/20    | Collection Technology, Inc.                           | Monterey Park     |         4 | Closure Permanent            |
| 611 |               | 05/18/20     | 03/21/20    | Collection Technology, Inc.                           | Rancho            |        51 | Layoff Permanent             |
| 612 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | San Diego         |        35 | Layoff Permanent             |
| 613 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | San Diego         |         1 | Layoff Permanent             |
| 614 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | San Diego         |         1 | Layoff Permanent             |
| 615 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | San Diego         |         1 | Layoff Permanent             |
| 616 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | San Diego         |         1 | Layoff Permanent             |
| 617 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | San Diego         |        16 | Layoff Permanent             |
| 618 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | San Diego         |         1 | Layoff Permanent             |
| 619 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | La Mesa           |         1 | Layoff Permanent             |
| 620 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | San Diego         |         1 | Layoff Permanent             |
| 621 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | Encinitas         |         1 | Layoff Permanent             |
| 622 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | Encinitas         |         2 | Layoff Permanent             |
| 623 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | La Jolla          |         4 | Layoff Permanent             |
| 624 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | La Jolla          |         3 | Layoff Permanent             |
| 625 |               | 05/20/20     | 03/21/20    | Scripps Health                                        | Chula Vista       |         1 | Layoff Permanent             |
| 626 |               | 06/30/20     | 03/22/20    | TransitAmerica Services, Inc.                         | Camp Pendleton    |        30 | Layoff Permanent             |
| 627 |               | 05/22/20     | 03/22/20    | Paris Precision, LLC                                  | Paso Robles       |       130 | Layoff Unknown at this time  |
| 628 |               | 03/22/20     | 03/22/20    | ZETA Communities                                      | Mcclellan         |       122 | Closure Permanent            |
| 629 |               | 03/10/20     | 03/23/20    | Mycom North America, Inc.                             | Santa Fe Springs  |       138 | Closure Permanent            |
| 630 |               | 05/20/20     | 03/23/20    | Boeing Company                                        | Huntington Beach  |        23 | Layoff Unknown at this time  |
| 631 |               | 05/20/20     | 03/23/20    | Boeing Company                                        | El Segundo        |        26 | Layoff Unknown at this time  |
| 632 |               | 05/27/20     | 03/23/20    | Rockwell Collins, Inc.                                | Poway             |         2 | Layoff Unknown at this time  |

Columns: ['Notice Date', 'Effective ', 'Received ', 'Company', 'City', 'No. Of ', 'Layoff/Closure']
Shape: (633, 7)

--- Concatenated DataFrame Group 2 ---


|    | Summary by  Month   |   Notices |  Employees  Affected   |   Permanent  Layoff |   Temporary  Layoff |   Not Identified  Layoff |   Permanent  Closure |   Temporary  Closure |   Not Identified Closure |
|---:|:--------------------|----------:|:-----------------------|--------------------:|--------------------:|-------------------------:|---------------------:|---------------------:|-------------------------:|
|  0 | July 2015           |        71 | 8,574                  |                  30 |                   2 |                       13 |                   25 |                    1 |                        0 |
|  1 | August 2015         |        69 | 5,890                  |                  28 |                   2 |                        7 |                   31 |                    0 |                        1 |
|  2 | September 2015      |        76 | 4,864                  |                  47 |                   2 |                       18 |                    7 |                    2 |                        0 |
|  3 | October 2015        |        71 | 6,102                  |                  38 |                   1 |                        8 |                   19 |                    2 |                        3 |
|  4 | November 2015       |        41 | 2,940                  |                  13 |                   0 |                       13 |                   10 |                    3 |                        2 |
|  5 | December 2015       |        69 | 8,729                  |                  21 |                   2 |                        7 |                   35 |                    2 |                        2 |
|  6 | January 2016        |        85 | 6,617                  |                  28 |                   0 |                        7 |                   47 |                    0 |                        3 |
|  7 | February 2016       |        92 | 6,899                  |                  52 |                   2 |                       12 |                   24 |                    1 |                        1 |
|  8 | March 2016          |        58 | 2,839                  |                  38 |                   0 |                        5 |                   14 |                    1 |                        0 |
|  9 | Total               |       632 | 53,454                 |                 295 |                  11 |                       90 |                  212 |                   12 |                       12 |

Columns: ['Summary by  Month', 'Notices', ' Employees  Affected', 'Permanent  Layoff', 'Temporary  Layoff', 'Not Identified  Layoff', 'Permanent  Closure', 'Temporary  Closure', 'Not Identified Closure']
Shape: (10, 9)


## 4. Full pipeline

In [42]:
def get_tables_from_sources(sources: list[str], output_path: str = None) -> dict[str, list[pd.DataFrame]]:
    processed_tables = {}
    conversions = convert_pdf_to_docling(sources)
    
    for source in sources:
        tables, full_prompt = get_tables_from_docling(conversions[source])
        json_instructions = generate(full_prompt)
        concatenated_dataframes = main_concatenation_logic(tables, json_instructions)
        processed_tables[source] = concatenated_dataframes
        if output_path:
            for i, df in enumerate(concatenated_dataframes):
                df.to_csv(f"{output_path}/{source}_{i}.csv", index=False)
    return processed_tables
    

In [43]:
sources = [
    "CA Warn Report.pdf",
    # "national-capitals.pdf",
    # "c935e2902adf7040a6ffe0db0f7c11e6.pdf",
]

output_path = "output"

processed_tables = get_tables_from_sources(sources)

Time taken: 312.5292
17 tables extracted
Input tokens: total_tokens=6506 cached_content_token_count=None
Output tokens: total_tokens=123 cached_content_token_count=None


In [47]:
for source in processed_tables:
    print(f"{source} has {len(processed_tables[source])} tables")

CA Warn Report.pdf has 2 tables
